# FER2013

In [ ]:
import matplotlib.pyplot as plt
import os
import numpy as np
import pandas as pd
import seaborn as sns
from PIL import Image
import tensorflow as tf
from numpy import asarray
import matplotlib.pyplot as plt
from IPython.display import SVG, Image
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import plot_model
from livelossplot.tf_keras import PlotLossesCallback
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.layers import Dense, Input, Dropout, Flatten, Conv2D, BatchNormalization, Activation, MaxPooling2D
import umap
from sklearn.preprocessing import StandardScaler
import os
import zipfile
from PIL import Image
from sklearn.preprocessing import LabelEncoder
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score, silhouette_score, normalized_mutual_info_score, davies_bouldin_score
from scipy.stats import multivariate_normal
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
    ### YES ###
df = pd.read_csv("fer2013.csv")
df.info()

In [ ]:
# Sample 15 random rows from the DataFrame
random_samples = df.sample(n=15, random_state=42)  # Set random_state for reproducibility
random_samples

In [6]:
# Loop through the sampled rows to display the images
for index, row in random_samples.iterrows():
    # Convert the pixel string into a NumPy array and reshape it
    pixels_array = np.array(list(map(int, row["pixels"].split())), dtype=np.uint8).reshape(48, 48)

In [ ]:
# Create a grid for 15 images (3 rows x 5 columns)
fig, axes = plt.subplots(3, 5, figsize=(15, 9))
axes = axes.flatten()  # Flatten the grid to easily iterate

# Loop through the sampled rows and display the images
for ax, (_, row) in zip(axes, random_samples.iterrows()):
    # Convert the pixel string into a NumPy array and reshape it
    pixels_array = np.array(list(map(int, row["pixels"].split())), dtype=np.uint8).reshape(48, 48)
    
    # Display the image
    ax.imshow(pixels_array, cmap="gray")
    # Add a title with relevant information
    ax.set_title(f"{row['Usage']}[{row.name}] = {row['emotion']}")
    ax.axis('off')  # Turn off axis for a cleaner look

# Adjust layout to avoid overlapping titles
plt.tight_layout()
plt.show()


In [8]:
# Split into training and testing sets based on 'Usage' column
train_fer2013 = df[df.Usage == "Training"]
test_fer2013 = df[df.Usage != "Training"]  # Assuming all non-training rows are for testing

In [ ]:
# Step 1: Convert 'pixels' column to numerical arrays
def convert_pixels(pixels_str):
    return np.array([int(pixel) for pixel in pixels_str.split()], dtype=np.uint8)

# Apply conversion
train_fer2013_pixels = np.vstack(train_fer2013["pixels"].apply(convert_pixels).values)
test_fer2013_pixels = np.vstack(test_fer2013["pixels"].apply(convert_pixels).values)

# Step 2: Reshape to flatten into 1D vectors (2304 features)
x_train_fer2013 = train_fer2013_pixels.reshape(-1, 48 * 48)  # Shape: (28   709, 2304)
x_test_fer2013 = test_fer2013_pixels.reshape(-1, 48 * 48)    # Shape: (7178, 2304)

# Step 3: Normalize the flattened data
scaler = StandardScaler()
x_train_fer2013_scaled = scaler.fit_transform(x_train_fer2013)
x_test_fer2013_scaled = scaler.transform(x_test_fer2013)

# Verify shapes
print("Shape of x_train_scaled:", x_train_fer2013_scaled.shape)  # Should be (28709, 2304)
print("Shape of x_test_scaled:", x_test_fer2013_scaled.shape)    # Should be (7178, 2304)

In [ ]:
# Extract labels directly as a 1D array
y_train_fer2013 = train_fer2013["emotion"].values  # Shape: (28709,)
y_test_fer2013 = test_fer2013["emotion"].values    # Shape: (7178,)

# Verify the shapes
print("Shape of y_train_fer2013:", y_train_fer2013.shape)
print("Shape of y_test_fer2013:", y_test_fer2013.shape)

In [ ]:
# Dataframe flatten and normalized
print("Shape of x_train_fer2013:", x_train_fer2013.shape)
print("Shape of x_test_fer2013:", x_test_fer2013.shape)
print("Shape of y_train_fer2013:", y_train_fer2013.shape)
print("Shape of y_test_fer2013:", y_test_fer2013.shape)

<!-- Supervised UMAP -->

<!-- ---------- -->

<!-- PCA + UMAP for FER2013 emotions -->

In [25]:
# # Step 1: Apply PCA to reduce dimensionality (e.g., 50 components)
# pca = PCA(0.85)
# x_train_pca = pca.fit_transform(x_train_fer2013)  # x_train is the normalized flattened pixel data

In [ ]:
# # Step 1: Apply UMAP for Dimensionality Reduction
# reducer = umap.UMAP(n_neighbors=15, min_dist=0.1, n_components=2, random_state=42)

# # Fit and transform UMAP on the training set
# x_train_pca_umap = reducer.fit_transform(x_train_pca)

In [ ]:
# # Step 2: Visualize the UMAP Results
# plt.figure(figsize=(10, 8))
# plt.scatter(x_train_pca_umap[:, 0], x_train_pca_umap[:, 1], c=y_train_fer2013, cmap="tab10", s=5, alpha=0.8,)
# plt.title("UMAP Projection of FER2013 Training Data")
# plt.xlabel("UMAP Component 1")
# plt.ylabel("UMAP Component 2")
# plt.colorbar(label="Emotion Labels")
# plt.show()

In [ ]:
# # Step 2: Calculate Silhouette Score
# sil_score_pca_umap = silhouette_score(x_train_pca_umap, y_train_fer2013)
# print(f"Silhouette Score: {sil_score_pca_umap:.4f}")

In [27]:
    ## CHECK NOW I HAVE CHANGED EVERYTHING OF THE PREPROCESSING ###


# Apply UMAP for dimensionality reduction (to 2D)
reducer = umap.UMAP(n_neighbors=15, min_dist=0.1, n_components=2, random_state=42)
X_umap = reducer.fit_transform(X_scaled)

# Add the UMAP results to the dataframe
df["umap_x"] = X_umap[:, 0]
df["umap_y"] = X_umap[:, 1]

In [ ]:
# # Visualize the results
# plt.figure(figsize=(10, 8))
# scatter = plt.scatter(df["umap_x"], df["umap_y"], c=df["emotion"], cmap="tab10", s=5, alpha=0.8)
# plt.colorbar(scatter, label="Emotion")
# plt.title("UMAP Projection of FER2013 Data")
# plt.xlabel("UMAP Component 1")
# plt.ylabel("UMAP Component 2")
# plt.show()

In [ ]:
# # Lower n_neighbors
# # Apply UMAP for dimensionality reduction (to 2D)
# reducer = umap.UMAP(n_neighbors=5, min_dist=0.1, n_components=2, random_state=42)
# x_umap_fer2013 = reducer.fit_transform(x_train_fer2013)

# # Add the UMAP results to the dataframe
# df["umap_x_5"] = X_umap_5[:, 0]
# df["umap_y_5"] = X_umap_5[:, 1]

In [ ]:
# # Visualize the results
# plt.figure(figsize=(10, 8))
# scatter = plt.scatter(df["umap_x_5"], df["umap_y_5"], c=df["emotion"], cmap="tab10", s=5, alpha=0.8)
# plt.colorbar(scatter, label="Emotion")
# plt.title("UMAP Projection of FER2013 Data")
# plt.xlabel("UMAP Component 1")
# plt.ylabel("UMAP Component 2")
# plt.show()

In [ ]:
# # Apply UMAP for dimensionality reduction (to 2D)
# reducer = umap.UMAP(metric='cosine', n_neighbors=5, min_dist=0.5, n_components=2, random_state=42)
# X_umap_5_05 = reducer.fit_transform(X_scaled)

# # Add the UMAP results to the dataframe
# df["umap_x_5_05"] = X_umap_5_05[:, 0]
# df["umap_y_5_05"] = X_umap_5_05[:, 1]

In [ ]:
# # Visualize the results
# plt.figure(figsize=(10, 8))
# scatter = plt.scatter(df["umap_x_5_05"], df["umap_y_5_05"], c=df["emotion"], cmap="tab10", s=5, alpha=0.8)
# plt.colorbar(scatter, label="Emotion")
# plt.title("UMAP Projection of FER2013 Data")
# plt.xlabel("UMAP Component 1")
# plt.ylabel("UMAP Component 2")
# plt.show()

In [ ]:
# # Apply UMAP for dimensionality reduction (to 2D)
# reducer = umap.UMAP(metric='cosine', n_neighbors=5, min_dist=0.5, n_components=2, random_state=42)
# X_umap_cosine_5_05 = reducer.fit_transform(X_scaled)

# # Add the UMAP results to the dataframe
# df["umap_x_cosine_5_05"] = X_umap_cosine_5_05[:, 0]
# df["umap_y_cosine_5_05"] = X_umap_cosine_5_05[:, 1]

In [ ]:
# # Visualize the results
# plt.figure(figsize=(10, 8))
# scatter = plt.scatter(df["umap_x_cosine_5_05"], df["umap_y_cosine_5_05"], c=df["emotion"], cmap="tab10", s=5, alpha=0.8)
# plt.colorbar(scatter, label="Emotion")
# plt.title("UMAP Projection of FER2013 Data")
# plt.xlabel("UMAP Component 1")
# plt.ylabel("UMAP Component 2")
# plt.show()

In [23]:
# from tensorflow.keras.applications import ResNet50
# from tensorflow.keras.applications.resnet50 import preprocess_input

-----------

PCA + UMAP 10 times

In [15]:
# Step 1: Apply PCA to reduce dimensionality (e.g., 50 components)
pca = PCA(0.95)
x_train_pca = pca.fit_transform(x_train_fer2013)  # x_train is the normalized flattened pixel data
np.save('x_train_pca.npy', x_train_pca)

In [ ]:
# UMAP Parameters
n_neighbors = 10
min_dist = 0.1
n_components = 2
n_runs = 10  # Number of UMAP runs
n_clusters = len(np.unique(y_train_fer2013))  # Number of clusters

# Store UMAP projections and KMeans results
pca_umap_projections = []
pca_ari_scores = []
pca_silhouette_scores = []

# Run UMAP + KMeans multiple times
for run in range(n_runs):
    # Step 1: Run UMAP
    umap_model = umap.UMAP(n_neighbors=n_neighbors, min_dist=min_dist, n_components=n_components, random_state=run)
    pca_projection = umap_model.fit_transform(x_train_pca, y_train_fer2013)
    pca_umap_projections.append(pca_projection)

    # Step 2: Apply KMeans
    kmeans = KMeans(n_clusters=n_clusters, random_state=run)
    predicted_labels = kmeans.fit_predict(pca_projection)

    # Step 3: Evaluate KMeans clustering
    ari = adjusted_rand_score(y_train_fer2013, predicted_labels)
    silhouette = silhouette_score(pca_projection, predicted_labels)
    
    pca_ari_scores.append(ari)
    pca_silhouette_scores.append(silhouette)

# Convert to numpy arrays for easy handling
pca_umap_projections = np.array(pca_umap_projections)
pca_mean_projection = np.mean(pca_umap_projections, axis=0)
pca_std_projection = np.std(pca_umap_projections, axis=0)

In [ ]:
# Step 4: Visualization
plt.figure(figsize=(12, 8))

# Plot all UMAP runs
for i, projection in enumerate(pca_umap_projections):
    plt.scatter(projection[:, 0], projection[:, 1], s=3, alpha=0.3, label=f"Run {i+1}")

# Plot the mean UMAP projection
plt.scatter(pca_mean_projection[:, 0], pca_mean_projection[:, 1], c='red', s=5, label="Mean Projection")

plt.title("PCA + Sup. UMAP Projections Across Multiple Runs")
plt.xlabel("UMAP Component 1")
plt.ylabel("UMAP Component 2")
plt.legend()
plt.show()

# # Step 5: Print Evaluation Results
# print(f"Mean ARI: {np.mean(ari_scores):.2f}, Std ARI: {np.std(ari_scores):.2f}")
# print(f"Mean Silhouette Score: {np.mean(silhouette_scores):.2f}, Std Silhouette Score: {np.std(silhouette_scores):.2f}")

In [ ]:
# Mean UMAP projection as input to KMeans to assign clusters

#  KMeans on the Mean Projection
kmeans_mean = KMeans(n_clusters=n_clusters, random_state=42)
pca_mean_projection_clusters = kmeans_mean.fit_predict(pca_mean_projection)

# Step 2: Visualize All Projections and the Mean Projection with Clusters
plt.figure(figsize=(12, 8))

# # Plot all UMAP runs as background points (semi-transparent)
# for i, projection in enumerate(umap_projections):
#     plt.scatter(projection[:, 0], projection[:, 1], s=3, alpha=0.1, color="gray", label=None)

# Plot the mean UMAP projection with cluster colors
plt.scatter(
    pca_mean_projection[:, 0],
    pca_mean_projection[:, 1],
    c=pca_mean_projection_clusters,
    cmap="tab10",
    s=10,
    alpha=0.8,
    label="Mean Projection (Clustered)"
)

# Add titles and labels
plt.title("PCA + UMAP + Kmeans on Mean Projection with Cluster Assignments")
plt.xlabel("UMAP Component 1")
plt.ylabel("UMAP Component 2")
plt.colorbar(label="Cluster Labels (KMeans)")
plt.show()

# # Step 3: Print Evaluation Results
# print(f"Mean ARI: {np.mean(ari_scores):.2f}, Std ARI: {np.std(ari_scores):.2f}")
# print(f"Mean Silhouette Score: {np.mean(silhouette_scores):.2f}, Std Silhouette Score: {np.std(silhouette_scores):.2f}")

In [ ]:
# Apply sup UMAP to PCA-reduced data
reducer_pca = umap.UMAP(n_neighbors=15, min_dist=0.1, n_components=2, random_state=42)
x_train_sup_umap_pca = reducer_pca.fit_transform(x_train_pca,y_train_fer2013)

In [ ]:
# Step 3: Visualize the results
plt.figure(figsize=(10, 8))
plt.scatter(
    x_train_sup_umap_pca[:, 0],
    x_train_sup_umap_pca[:, 1],
    c=y_train_fer2013,  # 0 for FER2013, 1 for RAF
    cmap="coolwarm",
    s=5,
    alpha=0.8
)
plt.title("PCA + Supervised UMAP Projection: Dataset Identification FER2013")
plt.xlabel("UMAP Component 1")
plt.ylabel("UMAP Component 2")
plt.colorbar(label= "Dataset FER2013" )
plt.show()

Gabor + PCA + Sup UMAP 10 runs

In [34]:
import cv2
import numpy as np

# Create Gabor Kernels
def create_gabor_kernels():
    """Generates a set of Gabor kernels with different orientations and frequencies."""
    kernels = []
    ksize = 31  # Kernel size
    sigma = 4.0  # Standard deviation of the Gaussian envelope
    lambd = 10.0  # Wavelength of the sinusoidal factor
    gamma = 0.5  # Spatial aspect ratio
    for theta in np.arange(0, np.pi, np.pi / 4):  # 8 orientations
        kernel = cv2.getGaborKernel((ksize, ksize), sigma, theta, lambd, gamma, psi=0, ktype=cv2.CV_32F)
        kernels.append(kernel)
    return kernels

# Apply Gabor Filters
def apply_gabor_filters(images, kernels):
    """Applies a set of Gabor filters to a batch of images."""
    gabor_features = []
    for image in images:
        image_2d = image.reshape(48, 48)  # Reshape back to 2D (assumes 48x48 images)
        responses = []
        for kernel in kernels:
            filtered = cv2.filter2D(image_2d, cv2.CV_32F, kernel)  # Apply Gabor filter
            responses.append(filtered.flatten())  # Flatten the filtered image
        gabor_features.append(np.concatenate(responses))  # Concatenate all filter responses
    return np.array(gabor_features)

In [ ]:
# Generate Gabor kernels
gabor_kernels = create_gabor_kernels()
print(f"Generated {len(gabor_kernels)} Gabor kernels.")

In [ ]:
# Apply Gabor filters to the training and test sets
x_train_gabor = apply_gabor_filters(x_train_fer2013, gabor_kernels)
x_test_gabor = apply_gabor_filters(x_test_fer2013, gabor_kernels)

print(f"Train Gabor feature shape: {x_train_gabor.shape}")
print(f"Test Gabor feature shape: {x_test_gabor.shape}")

In [37]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
x_train_gabor = scaler.fit_transform(x_train_gabor)
x_test_gabor = scaler.transform(x_test_gabor)

In [38]:
# Step 1: Apply PCA to reduce dimensionality (e.g., 50 components)
pca = PCA(0.95)
x_train_gabor_pca = pca.fit_transform(x_train_gabor)
np.save('x_train_pca.npy', x_train_gabor_pca)

In [ ]:
# UMAP Parameters
n_neighbors = 10
min_dist = 0.1
n_components = 2
n_runs = 10  # Number of UMAP runs
n_clusters = len(np.unique(y_train_fer2013))  # Number of clusters

# Store UMAP projections and KMeans results
gabor_pca_umap_projections = []
gabor_pca_ari_scores = []
gabor_pca_silhouette_scores = []

# Run UMAP + KMeans multiple times
for run in range(n_runs):
    # Step 1: Run UMAP
    umap_model = umap.UMAP(n_neighbors=n_neighbors, min_dist=min_dist, n_components=n_components, random_state=run)
    gabor_pca_projection = umap_model.fit_transform(x_train_gabor_pca, y_train_fer2013)
    gabor_pca_umap_projections.append(gabor_pca_projection)

    # Step 2: Apply KMeans
    kmeans = KMeans(n_clusters=n_clusters, random_state=run)
    predicted_labels = kmeans.fit_predict(gabor_pca_projection)

    # Step 3: Evaluate KMeans clustering
    ari = adjusted_rand_score(y_train_fer2013, predicted_labels)
    silhouette = silhouette_score(gabor_pca_projection, predicted_labels)
    
    pca_ari_scores.append(ari)
    pca_silhouette_scores.append(silhouette)

# Convert to numpy arrays for easy handling
gabor_pca_umap_projections = np.array(gabor_pca_umap_projections)
gabor_pca_mean_projection = np.mean(gabor_pca_umap_projections, axis=0)
gabor_pca_std_projection = np.std(gabor_pca_umap_projections, axis=0)

In [ ]:
# from sklearn.cluster import KMeans

# Apply KMeans to the UMAP-reduced data
n_clusters = len(np.unique(y_train_fer2013))  # Number of clusters = number of unique labels
kmeans = KMeans(n_clusters=n_clusters, random_state=42)
gabor_predicted_labels = kmeans.fit_predict(gabor_pca_mean_projection)

# Calculate Silhouette Score
sil_score = silhouette_score(gabor_pca_mean_projection, gabor_predicted_labels)
print(f"Silhouette Score (with KMeans): {sil_score:.2f}")

# Calculate Davies-Bouldin Index
db_score = davies_bouldin_score(gabor_pca_mean_projection, gabor_predicted_labels)
print(f"Davies-Bouldin Index (with KMeans): {db_score:.2f}")

In [ ]:
# Calculate Silhouette Score
sil_score2 = silhouette_score(gabor_pca_mean_projection, y_train_fer2013)
print(f"Silhouette Score: {sil_score2:.2f}")
# Calculate Davies-Bouldin Index
db_score2 = davies_bouldin_score(gabor_pca_mean_projection, y_train_fer2013)
print(f"Davies-Bouldin Index: {db_score2:.2f}")

In [ ]:
# t_student

from scipy.stats import ttest_ind

# Extract UMAP projections for two classes (e.g., class 0 and class 1)
class_0 = gabor_pca_mean_projection[y_train_fer2013 == 0]
class_1 = gabor_pca_mean_projection[y_train_fer2013 == 1]

# Perform t-test
t_stat, p_value = ttest_ind(class_0[:, 0], class_1[:, 0])  # Compare along UMAP component 1
print(f"T-test between Class 0 and Class 1: t-statistic={t_stat:.2f}, p-value={p_value:.4f}")


In [ ]:
from scipy.stats import multivariate_normal

# Fit Gaussian distributions to each class
unique_labels = np.unique(y_train_fer2013)
gaussian_params = {}

for label in unique_labels:
    class_points = gabor_pca_mean_projection[y_train_fer2013 == label]
    mean = np.mean(class_points, axis=0)
    cov = np.cov(class_points, rowvar=False)
    gaussian_params[label] = multivariate_normal(mean=mean, cov=cov)

# Example: Compare likelihoods of a random point belonging to each class
sample_point = gabor_pca_mean_projection[0]  # Random point
likelihoods = {label: dist.pdf(sample_point) for label, dist in gaussian_params.items()}
print("Likelihoods for Random Point:", likelihoods)


In [ ]:
plt.figure(figsize=(10, 8))
for label in np.unique(y_train_fer2013):
    class_points = gabor_pca_mean_projection[y_train_fer2013 == label]
    plt.scatter(class_points[:, 0], class_points[:, 1], label=f"Class {label}", alpha=0.7, s=5)

plt.title("UMAP Mean Projection with True Labels")
plt.xlabel("UMAP Component 1")
plt.ylabel("UMAP Component 2")
plt.legend()
plt.show()


Same metrics as before but on Kmeans results

In [ ]:
# Calculate Silhouette Score
sil_score2 = silhouette_score(gabor_pca_mean_projection, gabor_predicted_labels)
print(f"Silhouette Score: {sil_score2:.2f}")
# Calculate Davies-Bouldin Index
db_score2 = davies_bouldin_score(gabor_pca_mean_projection, gabor_predicted_labels)
print(f"Davies-Bouldin Index: {db_score2:.2f}")

In [ ]:
# t_student

from scipy.stats import ttest_ind

# Extract UMAP projections for two classes (e.g., class 0 and class 1)
class_0 = gabor_pca_mean_projection[gabor_predicted_labels == 0]
class_1 = gabor_pca_mean_projection[gabor_predicted_labels == 1]

# Perform t-test
t_stat, p_value = ttest_ind(class_0[:, 0], class_1[:, 0])  # Compare along UMAP component 1
print(f"T-test between Class 0 and Class 1: t-statistic={t_stat:.2f}, p-value={p_value:.4f}")


In [ ]:
from scipy.stats import multivariate_normal

# Fit Gaussian distributions to each class
unique_labels = np.unique(y_train_fer2013)
gaussian_params = {}

for label in unique_labels:
    class_points = gabor_predicted_labels[y_train_fer2013 == label]
    mean = np.mean(class_points, axis=0)
    cov = np.cov(class_points, rowvar=False)
    gaussian_params[label] = multivariate_normal(mean=mean, cov=cov)

# Example: Compare likelihoods of a random point belonging to each class
sample_point = gabor_predicted_labels[0]  # Random point
likelihoods = {label: dist.pdf(sample_point) for label, dist in gaussian_params.items()}
print("Likelihoods for Random Point:", likelihoods)


-----------

In [ ]:
# Step 1: Apply PCA to reduce dimensionality (e.g., 50 components)
pca = PCA(0.95)
x_train_pca = pca.fit_transform(x_train_fer2013)  # x_train is the normalized flattened pixel data
np.save('x_train_pca.npy', x_train_pca)

In [ ]:
# UMAP Parameters
n_neighbors = 10
min_dist = 0.1
n_components = 2
n_runs = 10  # Number of UMAP runs
n_clusters = len(np.unique(y_train_fer2013))  # Number of clusters

# Store UMAP projections and KMeans results
pca_umap_projections = []
pca_ari_scores = []
pca_silhouette_scores = []

# Run UMAP + KMeans multiple times
for run in range(n_runs):
    # Step 1: Run UMAP
    umap_model = umap.UMAP(n_neighbors=n_neighbors, min_dist=min_dist, n_components=n_components, random_state=run)
    pca_projection = umap_model.fit_transform(x_train_pca, y_train_fer2013)
    pca_umap_projections.append(pca_projection)

    # Step 2: Apply KMeans
    kmeans = KMeans(n_clusters=n_clusters, random_state=run)
    predicted_labels = kmeans.fit_predict(pca_projection)

    # Step 3: Evaluate KMeans clustering
    ari = adjusted_rand_score(y_train_fer2013, predicted_labels)
    silhouette = silhouette_score(pca_projection, predicted_labels)
    
    pca_ari_scores.append(ari)
    pca_silhouette_scores.append(silhouette)

# Convert to numpy arrays for easy handling
pca_umap_projections = np.array(pca_umap_projections)
pca_mean_projection = np.mean(pca_umap_projections, axis=0)
pca_std_projection = np.std(pca_umap_projections, axis=0)

## Dataset as label - Preparation

### RAF Import

In [12]:
class EmotionsDataloader(object):
    def __init__(self, image_dir, label_file):
        self.image_dir = image_dir
        self.label_file = label_file

    def read_images_labels(self):  
        train_images, train_labels = [], []
        test_images, test_labels = [], []

        # Read the label file and match labels to images
        with open(self.label_file, 'r') as file:
            for line in file:
                parts = line.strip().split()
                if len(parts) == 2:
                    image_name, label = parts[0], int(parts[1])
                    aligned_image_name = f"{image_name.split('.')[0]}_aligned.jpg"
                    image_path = os.path.join(self.image_dir, aligned_image_name)

                    if os.path.exists(image_path):
                        image = Image.open(image_path).convert('L')  # Convert to grayscale
                        image = image.resize((48, 48))  # Resize to 48x48

                        # Check if the label belongs to train or test set
                        if "train" in image_name:
                            train_images.append(np.array(image))
                            train_labels.append(label)
                        elif "test" in image_name:
                            test_images.append(np.array(image))
                            test_labels.append(label)
                    else:
                        print(f"Image not found: {aligned_image_name}")

        print(f"Loaded {len(train_images)} training images and {len(test_images)} test images.")
        print(f"Loaded {len(train_labels)} training labels and {len(test_labels)} test labels.")

        return (
            (np.array(train_images), np.array(train_labels)),
            (np.array(test_images), np.array(test_labels))
        )

    def load_data(self):
        return self.read_images_labels()

In [ ]:
# Set file paths
input_path = 'C:/Users/Lorenzo/OneDrive/Documents/DTU/Python/2024 Fall/MSc Thesis'
image_dir = os.path.join(input_path, 'extracted_data/Image/aligned')
label_file = os.path.join(input_path, 'extracted_data/EmoLabel/list_patition_label.txt')

# Instantiate and load the dataset
emotions_dataloader = EmotionsDataloader(image_dir, label_file)
(x_train_RAF, y_train_RAF), (x_test_RAF, y_test_RAF) = emotions_dataloader.load_data()

# Print dataset shapes
print(f"Training set shape: {x_train_RAF.shape}, {y_train_RAF.shape}")
print(f"Testing set shape: {x_test_RAF.shape}, {y_test_RAF.shape}")

# Display some random train and test images
def show_images(images, title_texts):
    cols = 5
    rows = int(len(images) / cols) + 1
    plt.figure(figsize=(15, 10))
    for i, (image, title) in enumerate(zip(images, title_texts)):
        plt.subplot(rows, cols, i + 1)
        plt.imshow(image, cmap=plt.cm.gray)
        plt.title(title, fontsize=12)
        plt.axis('off')

# Show some random train and test images
images_to_show = []
titles_to_show = []

for i in range(10):
    idx = np.random.randint(0, len(x_train_RAF))
    images_to_show.append(x_train_RAF[idx])
    titles_to_show.append(f"Train[{idx}] = {y_train_RAF[idx]}")

for i in range(5):
    idx = np.random.randint(0, len(x_test_RAF))
    images_to_show.append(x_test_RAF[idx])
    titles_to_show.append(f"Test[{idx}] = {y_test_RAF[idx]}")

show_images(images_to_show, titles_to_show)

In [ ]:
# Convert Training Data (RAF)
train_raf_df = pd.DataFrame({
    "emotion": y_train_RAF,
    "pixels": [" ".join(map(str, x.flatten())) for x in x_train_RAF],  # Flatten images and store as strings
    "Dataset": "RAF",
    "Usage": "Training"
})

# Convert Testing Data (RAF)
test_raf_df = pd.DataFrame({
    "emotion": y_test_RAF,
    "pixels": [" ".join(map(str, x.flatten())) for x in x_test_RAF],  # Flatten images and store as strings
    "Dataset": "RAF",
    "Usage": "Testing"
})

# Combine RAF Train and Test
raf_combined_df = pd.concat([train_raf_df, test_raf_df], ignore_index=True)

# Verify the structure of the RAF DataFrame
print(raf_combined_df.head())
print("RAF Combined Shape:", raf_combined_df.shape)

In [ ]:
# Combine RAF and FER2013 datasets
combined_df = pd.concat([raf_combined_df, df], ignore_index=True)

# Verify the combined DataFrame
print("Combined Dataset Shape:", combined_df.shape)
print(combined_df.head())

### Combining RAF & FER

In [ ]:
# Convert the 'pixels' column to numerical arrays for verification
combined_df["pixels"] = combined_df["pixels"].apply(lambda x: np.array(list(map(int, x.split()))))

# Check the shape of the first row to confirm it is flattened
print("Shape of a single image (pixels):", combined_df["pixels"].iloc[0].shape)  # Expected: (2304,)

In [ ]:
# Check mean and standard deviation of pixel values
pixel_values = np.vstack(combined_df["pixels"].values)
print("Mean of pixel values:", pixel_values.mean())
print("Standard deviation of pixel values:", pixel_values.std())

In [18]:
# Normalize the pixel values
scaler = StandardScaler()
normalized_pixels = scaler.fit_transform(pixel_values)  # Normalize pixel values

# Update the 'pixels' column with normalized values
combined_df["pixels"] = list(normalized_pixels)

In [ ]:
# Update 'Usage' column values to 'Testing' if they are 'PrivateTest' or 'PublicTest'
combined_df.loc[(combined_df["Usage"] == "PrivateTest") | (combined_df["Usage"] == "PublicTest"), "Usage"] = "Testing"

# Check if the update was successful
print(combined_df["Usage"].value_counts())


In [20]:
# Encode the 'Dataset' column as 0 (FER2013) and 1 (RAF)
combined_df["Dataset"] = combined_df["Dataset"].map({"FER2013": 0, "RAF": 1})

In [ ]:
print(combined_df["Dataset"].isna().sum())  # Check for NaN values in the 'Dataset' column

In [ ]:
print("Unique values in Dataset column after encoding:", combined_df["Dataset"].unique())

In [ ]:
# Split into training and testing sets
merged_train_data = combined_df[combined_df["Usage"] == "Training"]
merged_test_data = combined_df[combined_df["Usage"] != "Training"]

# Extract X (pixels) and y (Dataset) for training and testing
merged_x_train = np.vstack(merged_train_data["pixels"].values)  # Convert list of arrays to 2D array
merged_y_train = merged_train_data["Dataset"].values

merged_x_test = np.vstack(merged_test_data["pixels"].values)  # Convert list of arrays to 2D array
merged_y_test = merged_test_data["Dataset"].values

# Verify shapes
print("merged_x_train shape:", merged_x_train.shape)  # Expected: (num_train_samples, 2304)
print("merged_y_train shape:", merged_y_train.shape)  # Expected: (num_train_samples,)
print("merged_x_test shape:", merged_x_test.shape)    # Expected: (num_test_samples, 2304)
print("merged_y_test shape:", merged_y_test.shape)    # Expected: (num_test_samples,)

### RAF vs FER Sup UMAP

n_neighhours = 50

In [ ]:
# Define parameters
n_neighbors = 50
min_dist = 0.1
n_components = 2
n_runs = 10  # Number of runs

# Store UMAP projections for training and test sets
raf_fer_sup_umap_train_50_01 = []
raf_fer_sup_umap_test_50_01 = []

# Run UMAP multiple times
for run in range(n_runs):
    print(f"Running supervised UMAP - Iteration {run + 1}/{n_runs}...")

    # Create UMAP model
    umap_model = umap.UMAP(
        n_neighbors=n_neighbors, 
        min_dist=min_dist, 
        n_components=n_components, 
        random_state=run
    )

    # Fit and transform the training data with labels
    projection_train = umap_model.fit_transform(merged_x_train, merged_y_train)
    raf_fer_sup_umap_train_50_01.append(projection_train)
    
    # Transform the test data using the fitted model
    print(f"Running UMAP on Test Set - Iteration {run + 1}/{n_runs}...")
    projection_test = umap_model.transform(merged_x_test)
    raf_fer_sup_umap_test_50_01.append(projection_test)

# Convert the list of projections to numpy arrays
raf_fer_sup_umap_train_50_01 = np.array(raf_fer_sup_umap_train_50_01)
raf_fer_sup_umap_test_50_01 = np.array(raf_fer_sup_umap_test_50_01)

# Calculate mean and standard deviation of projections across runs (train and test)
raf_fer_mean_sup_umap_projection_train_50_01 = np.mean(raf_fer_sup_umap_train_50_01, axis=0)
raf_fer_std_sup_umap_projection_train_50_01 = np.std(raf_fer_sup_umap_train_50_01, axis=0)

raf_fer_mean_sup_umap_projection_test_50_01 = np.mean(raf_fer_sup_umap_test_50_01, axis=0)
raf_fer_std_sup_umap_projection_test_50_01 = np.std(raf_fer_sup_umap_test_50_01, axis=0)

# Save the projections, mean, and standard deviation for the training set
np.save('raf_fer_sup_umap_train_50_01.npy', raf_fer_sup_umap_train_50_01)
np.save('raf_fer_mean_sup_umap_projection_train_50_01.npy', raf_fer_mean_sup_umap_projection_train_50_01)
np.save('raf_fer_std_sup_umap_projection_train_50_01.npy', raf_fer_std_sup_umap_projection_train_50_01)

# Save the projections, mean, and standard deviation for the test set
np.save('raf_fer_sup_umap_test_50_01.npy', raf_fer_sup_umap_test_50_01)
np.save('raf_fer_mean_sup_umap_projection_test_50_01.npy', raf_fer_mean_sup_umap_projection_test_50_01)
np.save('raf_fer_std_sup_umap_projection_test_50_01.npy', raf_fer_std_sup_umap_projection_test_50_01)

# Output confirmation
print("supervised UMAP projections for training and test sets, mean, and standard deviations have been saved.")

In [ ]:
from matplotlib.colors import ListedColormap

# Define a custom colormap with green and blue
custom_cmap = ListedColormap(["lightgreen", "pink"])

# Create the scatter plot
plt.figure(figsize=(10, 8))
scatter = plt.scatter(
    raf_fer_mean_sup_umap_projection_train_50_01[:, 0],
    raf_fer_mean_sup_umap_projection_train_50_01[:, 1],
    c=merged_y_train,
    cmap=custom_cmap,
    s=5,
    alpha=0.8
)

# Add title and labels
plt.title("n_neighbours=50 Supervised UMAP Projection of RAFDB-FER2013 Merged Training Data (10 Runs)")
plt.xlabel("UMAP Component 1")
plt.ylabel("UMAP Component 2")

# Add and configure the colorbar
cbar = plt.colorbar(scatter)
cbar.set_ticks([0, 1])  # Ensure ticks align with both labels
cbar.set_ticklabels(["Original source 0 - (FER2013)", "Original source 1 - (RAF-DB)"])  # Customize labels
cbar.set_label("Image Original Source Labels")

plt.show()

In [ ]:
# ARI
ari_raf_fer_umap_sup_50_01 = adjusted_rand_score(merged_y_test, KNeighborsClassifier(n_neighbors=1).fit(raf_fer_mean_sup_umap_projection_train_50_01, merged_y_train).predict(raf_fer_mean_sup_umap_projection_test_50_01)) # second argument is y_test_pred_pca
print(f"ARI: {ari_raf_fer_umap_sup_50_01:.4f}")
# Silhouette Score
silhouette_raf_fer_umap_sup_50_01 = silhouette_score(raf_fer_mean_sup_umap_projection_test_50_01, KNeighborsClassifier(n_neighbors=1).fit(raf_fer_mean_sup_umap_projection_train_50_01, merged_y_train).predict(raf_fer_mean_sup_umap_projection_test_50_01))
print(f"Silhouette Score: {silhouette_raf_fer_umap_sup_50_01:.2f}")
# Use KMeans for clustering
n_clusters = len(np.unique(merged_y_train))  # Number of clusters = number of unique labels
kmeans = KMeans(n_clusters=n_clusters, random_state=42)

# Fit KMeans on the training set UMAP projections
predicted_labels = kmeans.fit_predict(raf_fer_mean_sup_umap_projection_train_50_01)

# Compute Davies-Bouldin Index
raf_fer_sup_umap_projection_50_01_db_score = davies_bouldin_score(
    raf_fer_mean_sup_umap_projection_train_50_01,  # Training set mean projections
    predicted_labels  # Labels from KMeans clustering
)

print(f"Davies-Bouldin Index: {raf_fer_sup_umap_projection_50_01_db_score:.2f}")

#### n_neighbours = 10

In [24]:
# load the projections, mean, and standard deviation for the training set
raf_fer_sup_umap_train_10_01= np.load('raf_fer_sup_umap_train_10_01.npy')
raf_fer_mean_sup_umap_projection_train_10_01= np.load('raf_fer_mean_sup_umap_projection_train_10_01.npy')
raf_fer_std_sup_umap_projection_train_10_01= np.load('raf_fer_std_sup_umap_projection_train_10_01.npy')

# load the projections, mean, and standard deviation for the test set
raf_fer_sup_umap_test_10_01= np.load('raf_fer_sup_umap_test_10_01.npy')
raf_fer_mean_sup_umap_projection_test_10_01= np.load('raf_fer_mean_sup_umap_projection_test_10_01.npy')
raf_fer_std_sup_umap_projection_test_10_01= np.load('raf_fer_std_sup_umap_projection_test_10_01.npy')

In [ ]:
# Define parameters
n_neighbors = 10
min_dist = 0.1
n_components = 2
n_runs = 10  # Number of runs

# Store UMAP projections for training and test sets
raf_fer_sup_umap_train_10_01 = []
raf_fer_sup_umap_test_10_01 = []

# Run UMAP multiple times
for run in range(n_runs):
    print(f"Running Supervised UMAP - Iteration {run + 1}/{n_runs}...")

    # Create UMAP model
    umap_model = umap.UMAP(
        n_neighbors=n_neighbors, 
        min_dist=min_dist, 
        n_components=n_components, 
        random_state=run
    )

    # Fit and transform the training data with labels
    projection_train = umap_model.fit_transform(merged_x_train, merged_y_train)
    raf_fer_sup_umap_train_10_01.append(projection_train)
    
    # Transform the test data using the fitted model
    print(f"Running UMAP on Test Set - Iteration {run + 1}/{n_runs}...")
    projection_test = umap_model.transform(merged_x_test)
    raf_fer_sup_umap_test_10_01.append(projection_test)

# Convert the list of projections to numpy arrays
raf_fer_sup_umap_train_10_01 = np.array(raf_fer_sup_umap_train_10_01)
raf_fer_sup_umap_test_10_01 = np.array(raf_fer_sup_umap_test_10_01)

# Calculate mean and standard deviation of projections across runs (train and test)
raf_fer_mean_sup_umap_projection_train_10_01 = np.mean(raf_fer_sup_umap_train_10_01, axis=0)
raf_fer_std_sup_umap_projection_train_10_01 = np.std(raf_fer_sup_umap_train_10_01, axis=0)

raf_fer_mean_sup_umap_projection_test_10_01 = np.mean(raf_fer_sup_umap_test_10_01, axis=0)
raf_fer_std_sup_umap_projection_test_10_01 = np.std(raf_fer_sup_umap_test_10_01, axis=0)

# Save the projections, mean, and standard deviation for the training set
np.save('raf_fer_sup_umap_train_10_01.npy', raf_fer_sup_umap_train_10_01)
np.save('raf_fer_mean_sup_umap_projection_train_10_01.npy', raf_fer_mean_sup_umap_projection_train_10_01)
np.save('raf_fer_std_sup_umap_projection_train_10_01.npy', raf_fer_std_sup_umap_projection_train_10_01)

# Save the projections, mean, and standard deviation for the test set
np.save('raf_fer_sup_umap_test_10_01.npy', raf_fer_sup_umap_test_10_01)
np.save('raf_fer_mean_sup_umap_projection_test_10_01.npy', raf_fer_mean_sup_umap_projection_test_10_01)
np.save('raf_fer_std_sup_umap_projection_test_10_01.npy', raf_fer_std_sup_umap_projection_test_10_01)

# Output confirmation
print("Supervised UMAP projections for training and test sets, mean, and standard deviations have been saved.")

In [ ]:
from matplotlib.colors import ListedColormap

# Define a custom colormap with green and blue
custom_cmap = ListedColormap(["lightgreen", "blue"])

# Create the scatter plot
plt.figure(figsize=(10, 8))
scatter = plt.scatter(
    raf_fer_mean_sup_umap_projection_train_10_01[:, 0],
    raf_fer_mean_sup_umap_projection_train_10_01[:, 1],
    c=merged_y_train,
    cmap=custom_cmap,
    s=5,
    alpha=0.8
)

# Add title and labels
plt.title("Supervised UMAP Projection of RAFDB-FER2013 Merged Training Data (10 Runs)")
plt.xlabel("UMAP Component 1")
plt.ylabel("UMAP Component 2")

# Add and configure the colorbar
cbar = plt.colorbar(scatter)
cbar.set_ticks([0, 1])  # Ensure ticks align with both labels
cbar.set_ticklabels(["Original source 0 - (FER2013)", "Original source 1 - (RAF-DB)"])  # Customize labels
cbar.set_label("Image Original Source Labels")

plt.show()


In [ ]:
# ARI
ari_raf_fer_umap_sup_10_01 = adjusted_rand_score(merged_y_test, KNeighborsClassifier(n_neighbors=1).fit(raf_fer_mean_sup_umap_projection_train_10_01, merged_y_train).predict(raf_fer_mean_sup_umap_projection_test_10_01)) # second argument is y_test_pred_pca
print(f"ARI: {ari_raf_fer_umap_sup_10_01:.4f}")
# Silhouette Score
silhouette_raf_fer_umap_sup_10_01 = silhouette_score(raf_fer_mean_sup_umap_projection_test_10_01, KNeighborsClassifier(n_neighbors=1).fit(raf_fer_mean_sup_umap_projection_train_10_01, merged_y_train).predict(raf_fer_mean_sup_umap_projection_test_10_01))
print(f"Silhouette Score: {silhouette_raf_fer_umap_sup_10_01:.2f}")
# Use KMeans for clustering
n_clusters = len(np.unique(merged_y_train))  # Number of clusters = number of unique labels
kmeans = KMeans(n_clusters=n_clusters, random_state=42)

# Fit KMeans on the training set UMAP projections
predicted_labels = kmeans.fit_predict(raf_fer_mean_sup_umap_projection_train_10_01)

# Compute Davies-Bouldin Index
raf_fer_sup_umap_projection_10_01_db_score = davies_bouldin_score(
    raf_fer_mean_sup_umap_projection_train_10_01,  # Training set mean projections
    predicted_labels  # Labels from KMeans clustering
)

print(f"Davies-Bouldin Index: {raf_fer_sup_umap_projection_10_01_db_score:.2f}")

-------

### RAF vs FER Unsup UMAP

n_neighbors = 50

In [ ]:
# Define parameters
n_neighbors = 50
min_dist = 0.1
n_components = 2
n_runs = 10  # Number of runs

# Store UMAP projections for training and test sets
raf_fer_unsup_umap_train_50_01 = []
raf_fer_unsup_umap_test_50_01 = []

# Run UMAP multiple times
for run in range(n_runs):
    print(f"Running unsupervised UMAP - Iteration {run + 1}/{n_runs}...")

    # Create UMAP model
    umap_model = umap.UMAP(
        n_neighbors=n_neighbors, 
        min_dist=min_dist, 
        n_components=n_components, 
        random_state=run
    )

    # Fit and transform the training data with labels
    projection_train = umap_model.fit_transform(merged_x_train)
    raf_fer_unsup_umap_train_50_01.append(projection_train)
    
    # Transform the test data using the fitted model
    print(f"Running UMAP on Test Set - Iteration {run + 1}/{n_runs}...")
    projection_test = umap_model.transform(merged_x_test)
    raf_fer_unsup_umap_test_50_01.append(projection_test)

# Convert the list of projections to numpy arrays
raf_fer_unsup_umap_train_50_01 = np.array(raf_fer_unsup_umap_train_50_01)
raf_fer_unsup_umap_test_50_01 = np.array(raf_fer_unsup_umap_test_50_01)

# Calculate mean and standard deviation of projections across runs (train and test)
raf_fer_mean_unsup_umap_projection_train_50_01 = np.mean(raf_fer_unsup_umap_train_50_01, axis=0)
raf_fer_std_unsup_umap_projection_train_50_01 = np.std(raf_fer_unsup_umap_train_50_01, axis=0)

raf_fer_mean_unsup_umap_projection_test_50_01 = np.mean(raf_fer_unsup_umap_test_50_01, axis=0)
raf_fer_std_unsup_umap_projection_test_50_01 = np.std(raf_fer_unsup_umap_test_50_01, axis=0)

# Save the projections, mean, and standard deviation for the training set
np.save('raf_fer_unsup_umap_train_50_01.npy', raf_fer_unsup_umap_train_50_01)
np.save('raf_fer_mean_unsup_umap_projection_train_50_01.npy', raf_fer_mean_unsup_umap_projection_train_50_01)
np.save('raf_fer_std_unsup_umap_projection_train_50_01.npy', raf_fer_std_unsup_umap_projection_train_50_01)

# Save the projections, mean, and standard deviation for the test set
np.save('raf_fer_unsup_umap_test_50_01.npy', raf_fer_unsup_umap_test_50_01)
np.save('raf_fer_mean_unsup_umap_projection_test_50_01.npy', raf_fer_mean_unsup_umap_projection_test_50_01)
np.save('raf_fer_std_unsup_umap_projection_test_50_01.npy', raf_fer_std_unsup_umap_projection_test_50_01)

# Output confirmation
print("unsupervised UMAP projections for training and test sets, mean, and standard deviations have been saved.")

In [ ]:
from matplotlib.colors import ListedColormap

# Define a custom colormap with green and blue
custom_cmap = ListedColormap(["lightgreen", "pink"])

# Create the scatter plot
plt.figure(figsize=(10, 8))
scatter = plt.scatter(
    raf_fer_mean_unsup_umap_projection_train_50_01[:, 0],
    raf_fer_mean_unsup_umap_projection_train_50_01[:, 1],
    c=merged_y_train,
    cmap=custom_cmap,
    s=5,
    alpha=0.8
)

# Add title and labels
plt.title("n_neighbours=50 Unsupervised UMAP Projection of RAFDB-FER2013 Merged Training Data (10 Runs)")
plt.xlabel("UMAP Component 1")
plt.ylabel("UMAP Component 2")

# Add and configure the colorbar
cbar = plt.colorbar(scatter)
cbar.set_ticks([0, 1])  # Ensure ticks align with both labels
cbar.set_ticklabels(["Original source 0 - (FER2013)", "Original source 1 - (RAF-DB)"])  # Customize labels
cbar.set_label("Image Original Source Labels")

plt.show()

In [ ]:
# ARI
ari_raf_fer_umap_unsup_50_01 = adjusted_rand_score(merged_y_test, KNeighborsClassifier(n_neighbors=1).fit(raf_fer_mean_unsup_umap_projection_train_50_01, merged_y_train).predict(raf_fer_mean_unsup_umap_projection_test_50_01)) # second argument is y_test_pred_pca
print(f"ARI: {ari_raf_fer_umap_unsup_50_01:.4f}")
# Silhouette Score
silhouette_raf_fer_umap_unsup_50_01 = silhouette_score(raf_fer_mean_unsup_umap_projection_test_50_01, KNeighborsClassifier(n_neighbors=1).fit(raf_fer_mean_unsup_umap_projection_train_50_01, merged_y_train).predict(raf_fer_mean_unsup_umap_projection_test_50_01))
print(f"Silhouette Score: {silhouette_raf_fer_umap_unsup_50_01:.2f}")
# Use KMeans for clustering
n_clusters = len(np.unique(merged_y_train))  # Number of clusters = number of unique labels
kmeans = KMeans(n_clusters=n_clusters, random_state=42)

# Fit KMeans on the training set UMAP projections
predicted_labels = kmeans.fit_predict(raf_fer_mean_unsup_umap_projection_train_50_01)

# Compute Davies-Bouldin Index
raf_fer_unsup_umap_projection_50_01_db_score = davies_bouldin_score(
    raf_fer_mean_unsup_umap_projection_train_50_01,  # Training set mean projections
    predicted_labels  # Labels from KMeans clustering
)

print(f"Davies-Bouldin Index: {raf_fer_unsup_umap_projection_50_01_db_score:.2f}")

#### n_neighbours= 10

In [26]:
# load the projections, mean, and standard deviation for the training set
raf_fer_unsup_umap_train_10_01= np.load('raf_fer_unsup_umap_train_10_01.npy')
raf_fer_mean_unsup_umap_projection_train_10_01= np.load('raf_fer_mean_unsup_umap_projection_train_10_01.npy')
raf_fer_std_unsup_umap_projection_train_10_01= np.load('raf_fer_std_unsup_umap_projection_train_10_01.npy')

# load the projections, mean, and standard deviation for the test set
raf_fer_unsup_umap_test_10_01= np.load('raf_fer_unsup_umap_test_10_01.npy')
raf_fer_mean_unsup_umap_projection_test_10_01= np.load('raf_fer_mean_unsup_umap_projection_test_10_01.npy')
raf_fer_std_unsup_umap_projection_test_10_01= np.load('raf_fer_std_unsup_umap_projection_test_10_01.npy')

In [ ]:
# Define parameters
n_neighbors = 10
min_dist = 0.1
n_components = 2
n_runs = 10  # Number of runs

# Store UMAP projections for training and test sets
raf_fer_unsup_umap_train_10_01 = []
raf_fer_unsup_umap_test_10_01 = []

# Run UMAP multiple times
for run in range(n_runs):
    print(f"Running unsupervised UMAP - Iteration {run + 1}/{n_runs}...")

    # Create UMAP model
    umap_model = umap.UMAP(
        n_neighbors=n_neighbors, 
        min_dist=min_dist, 
        n_components=n_components, 
        random_state=run
    )

    # Fit and transform the training data with labels
    projection_train = umap_model.fit_transform(merged_x_train)
    raf_fer_unsup_umap_train_10_01.append(projection_train)
    
    # Transform the test data using the fitted model
    print(f"Running UMAP on Test Set - Iteration {run + 1}/{n_runs}...")
    projection_test = umap_model.transform(merged_x_test)
    raf_fer_unsup_umap_test_10_01.append(projection_test)

# Convert the list of projections to numpy arrays
raf_fer_unsup_umap_train_10_01 = np.array(raf_fer_unsup_umap_train_10_01)
raf_fer_unsup_umap_test_10_01 = np.array(raf_fer_unsup_umap_test_10_01)

# Calculate mean and standard deviation of projections across runs (train and test)
raf_fer_mean_unsup_umap_projection_train_10_01 = np.mean(raf_fer_unsup_umap_train_10_01, axis=0)
raf_fer_std_unsup_umap_projection_train_10_01 = np.std(raf_fer_unsup_umap_train_10_01, axis=0)

raf_fer_mean_unsup_umap_projection_test_10_01 = np.mean(raf_fer_unsup_umap_test_10_01, axis=0)
raf_fer_std_unsup_umap_projection_test_10_01 = np.std(raf_fer_unsup_umap_test_10_01, axis=0)

# Save the projections, mean, and standard deviation for the training set
np.save('raf_fer_unsup_umap_train_10_01.npy', raf_fer_unsup_umap_train_10_01)
np.save('raf_fer_mean_unsup_umap_projection_train_10_01.npy', raf_fer_mean_unsup_umap_projection_train_10_01)
np.save('raf_fer_std_unsup_umap_projection_train_10_01.npy', raf_fer_std_unsup_umap_projection_train_10_01)

# Save the projections, mean, and standard deviation for the test set
np.save('raf_fer_unsup_umap_test_10_01.npy', raf_fer_unsup_umap_test_10_01)
np.save('raf_fer_mean_unsup_umap_projection_test_10_01.npy', raf_fer_mean_unsup_umap_projection_test_10_01)
np.save('raf_fer_std_unsup_umap_projection_test_10_01.npy', raf_fer_std_unsup_umap_projection_test_10_01)

# Output confirmation
print("unsupervised UMAP projections for training and test sets, mean, and standard deviations have been saved.")

In [ ]:
from matplotlib.colors import ListedColormap

# Define a custom colormap with green and blue
custom_cmap = ListedColormap(["lightgreen", "blue"])

# Create the scatter plot
plt.figure(figsize=(10, 8))
scatter = plt.scatter(
    raf_fer_mean_unsup_umap_projection_train_10_01[:, 0],
    raf_fer_mean_unsup_umap_projection_train_10_01[:, 1],
    c=merged_y_train,
    cmap=custom_cmap,
    s=5,
    alpha=0.8
)

# Add title and labels
plt.title("Unsupervised UMAP Projection of RAFDB-FER2013 Merged Training Data (10 Runs)")
plt.xlabel("UMAP Component 1")
plt.ylabel("UMAP Component 2")

# Add and configure the colorbar
cbar = plt.colorbar(scatter)
cbar.set_ticks([0, 1])  # Ensure ticks align with both labels
cbar.set_ticklabels(["Original source 0 - (FER2013)", "Original source 1 - (RAF-DB)"])  # Customize labels
cbar.set_label("Image Original Source Labels")

plt.show()

In [ ]:
# ARI
ari_raf_fer_umap_unsup_10_01 = adjusted_rand_score(merged_y_test, KNeighborsClassifier(n_neighbors=1).fit(raf_fer_mean_unsup_umap_projection_train_10_01, merged_y_train).predict(raf_fer_mean_unsup_umap_projection_test_10_01)) # second argument is y_test_pred_pca
print(f"ARI: {ari_raf_fer_umap_unsup_10_01:.4f}")
# Silhouette Score
silhouette_raf_fer_umap_unsup_10_01 = silhouette_score(raf_fer_mean_unsup_umap_projection_test_10_01, KNeighborsClassifier(n_neighbors=1).fit(raf_fer_mean_unsup_umap_projection_train_10_01, merged_y_train).predict(raf_fer_mean_unsup_umap_projection_test_10_01))
print(f"Silhouette Score: {silhouette_raf_fer_umap_unsup_10_01:.2f}")
# Use KMeans for clustering
n_clusters = len(np.unique(merged_y_train))  # Number of clusters = number of unique labels
kmeans = KMeans(n_clusters=n_clusters, random_state=42)

# Fit KMeans on the training set UMAP projections
predicted_labels = kmeans.fit_predict(raf_fer_mean_unsup_umap_projection_train_10_01)

# Compute Davies-Bouldin Index
raf_fer_unsup_umap_projection_10_01_db_score = davies_bouldin_score(
    raf_fer_mean_unsup_umap_projection_train_10_01,  # Training set mean projections
    predicted_labels  # Labels from KMeans clustering
)

print(f"Davies-Bouldin Index: {raf_fer_unsup_umap_projection_10_01_db_score:.2f}")

In [ ]:
# Define the custom colormap with green and blue
custom_cmap = ListedColormap(["lightgreen", "blue"])

# Create a figure with two subplots side by side
fig, axes = plt.subplots(1, 2, figsize=(20, 8))

# Plot the unsupervised UMAP
scatter1 = axes[0].scatter(
    raf_fer_mean_unsup_umap_projection_train_10_01[:, 0],
    raf_fer_mean_unsup_umap_projection_train_10_01[:, 1],
    c=merged_y_train,
    cmap=custom_cmap,
    s=5,
    alpha=0.8
)
axes[0].set_title("Unsupervised UMAP Projection of RAFDB-FER2013 Merged Training Data (10 Runs)")
axes[0].set_xlabel("UMAP Component 1")
axes[0].set_ylabel("UMAP Component 2")
# cbar1 = fig.colorbar(scatter1, ax=axes[0], orientation="vertical")
# cbar1.set_ticks([0, 1])  # Ensure ticks align with both labels
# cbar1.set_ticklabels(["Original source 0 - (FER2013)", "Original source 1 - (RAF-DB)"])  # Customize labels
# cbar1.set_label("Image Original Source Labels")

# Plot the supervised UMAP
scatter2 = axes[1].scatter(
    raf_fer_mean_sup_umap_projection_train_10_01[:, 0],
    raf_fer_mean_sup_umap_projection_train_10_01[:, 1],
    c=merged_y_train,
    cmap=custom_cmap,
    s=5,
    alpha=0.8
)
axes[1].set_title("Supervised UMAP Projection of RAFDB-FER2013 Merged Training Data (10 Runs)")
axes[1].set_xlabel("UMAP Component 1")
axes[1].set_ylabel("UMAP Component 2")
cbar2 = fig.colorbar(scatter2, ax=axes[1], orientation="vertical")
cbar2.set_ticks([0, 1])  # Ensure ticks align with both labels
cbar2.set_ticklabels(["Original source 0 - (FER2013)", "Original source 1 - (RAF-DB)"])  # Customize labels
cbar2.set_label("Image Original Source Labels")

# Adjust layout
plt.tight_layout()
plt.show()

---------

---------

In [ ]:
# # Define parameters
# n_neighbors = 10
# min_dist = 0.1
# n_components = 2
# n_runs = 10  # Number of runs

# # Store UMAP projections for each run
# RAF_FER_sup_umap_projections_10_01 = []

# # Run UMAP multiple times
# for run in range(n_runs):
#     # Create UMAP model
#     umap_model = umap.UMAP(n_neighbors=n_neighbors, min_dist=min_dist, n_components=n_components,random_state=None)
    
#     # Fit and transform the data
#     RAF_FER_projection = umap_model.fit_transform(x_train,y_train)
    
#     # Store the projection
#     RAF_FER_sup_umap_projections_10_01.append(RAF_FER_projection)

# # Convert the list of projections to a numpy array
# RAF_FER_sup_umap_projections_10_01 = np.array(RAF_FER_sup_umap_projections_10_01)

# # Calculate mean and standard deviation of projections across runs
# RAF_FER_mean_sup_umap_projection_10_01 = np.mean(RAF_FER_sup_umap_projections_10_01, axis=0)
# RAF_FER_std_sup_umap_projection_10_01 = np.std(RAF_FER_sup_umap_projections_10_01, axis=0)

# # Save the projections, mean, and standard deviation
# np.save('RAF_FER_sup_umap_projections_10_01.npy', RAF_FER_sup_umap_projections_10_01)
# np.save('RAF_FER_mean_sup_projection_10_01_35.npy', RAF_FER_mean_sup_umap_projection_10_01)
# np.save('RAF_FER_std_sup_projection_10_01_35.npy', RAF_FER_std_sup_umap_projection_10_01)

# # Output confirmation
# print("UMAP projections, mean, and standard deviation have been saved with identifiers '_10_01'.")

In [ ]:
# Visualize the UMAP results
plt.figure(figsize=(10, 8))
plt.scatter(RAF_FER_mean_sup_umap_projection_10_01[:, 0], RAF_FER_mean_sup_umap_projection_10_01[:, 1], c=y_train, cmap="coolwarm", s=5, alpha=0.8)
plt.title("Sup. UMAP Projection: Dataset Identification (RAF vs FER2013)")
plt.xlabel("UMAP Component 1")
plt.ylabel("UMAP Component 2")
plt.colorbar(label="Dataset (0=FER2013, 1=RAF)")
plt.show()

In [ ]:
# Calculate Silhouette Score
RAF_FER_sil_score = silhouette_score(RAF_FER_mean_sup_umap_projection_10_01, y_train)
print(f"Silhouette Score: {RAF_FER_sil_score:.2f}")
# Calculate Davies-Bouldin Index
RAF_FER_db_score = davies_bouldin_score(RAF_FER_mean_sup_umap_projection_10_01, y_train)
print(f"Davies-Bouldin Index: {RAF_FER_db_score:.2f}")

In [ ]:
# t_student

from scipy.stats import ttest_ind

# Extract UMAP projections for two classes (e.g., class 0 and class 1)
class_0 = RAF_FER_mean_sup_umap_projection_10_01[y_train == 0]
class_1 = RAF_FER_mean_sup_umap_projection_10_01[y_train == 1]

# Perform t-test
t_stat, p_value = ttest_ind(class_0[:, 0], class_1[:, 0])  # Compare along UMAP component 1
print(f"T-test between Class 0 and Class 1: t-statistic={t_stat:.2f}, p-value={p_value:.4f}")


In [ ]:
## NOT RUN YET ##

from scipy.stats import multivariate_normal

# Fit Gaussian distributions to each class
unique_labels = np.unique(y_train)
gaussian_params = {}

for label in unique_labels:
    class_points = gabor_predicted_labels[y_train == label]
    mean = np.mean(class_points, axis=0)
    cov = np.cov(class_points, rowvar=False)
    gaussian_params[label] = multivariate_normal(mean=mean, cov=cov)

# Example: Compare likelihoods of a random point belonging to each class
sample_point = gabor_predicted_labels[0]  # Random point
likelihoods = {label: dist.pdf(sample_point) for label, dist in gaussian_params.items()}
print("Likelihoods for Random Point:", likelihoods)


In [33]:
from sklearn.metrics import silhouette_score, davies_bouldin_score

In [ ]:
from sklearn.cluster import KMeans

# Apply KMeans to the UMAP-reduced data
n_clusters = len(np.unique(y_train))  # Number of clusters = number of unique labels
kmeans = KMeans(n_clusters=n_clusters, random_state=42)
predicted_labels = kmeans.fit_predict(x_train_sup_umap)

# Calculate Silhouette Score
sil_score = silhouette_score(x_train_sup_umap, predicted_labels)
print(f"Silhouette Score (with KMeans): {sil_score:.2f}")

# Calculate Davies-Bouldin Index
db_score = davies_bouldin_score(x_train_sup_umap, predicted_labels)
print(f"Davies-Bouldin Index (with KMeans): {db_score:.2f}")

In [ ]:
from sklearn.metrics import adjusted_rand_score, normalized_mutual_info_score

ari = adjusted_rand_score(y_train, predicted_labels)
print(f"Adjusted Rand Index (ARI): {ari:.2f}")

nmi = normalized_mutual_info_score(y_train, predicted_labels)
print(f"Normalized Mutual Information (NMI): {nmi:.2f}")

 Unsupervised UMAP only

In [ ]:
# Define parameters
n_neighbors = 10
min_dist = 0.1
n_components = 2
n_runs = 10  # Number of runs

# Store UMAP projections for each run
RAF_FER_unsup_umap_projections_10_01 = []

# Run UMAP multiple times
for run in range(n_runs):
    # Create UMAP model
    umap_model = umap.UMAP(n_neighbors=n_neighbors, min_dist=min_dist, n_components=n_components, random_state=None)
    
    # Fit and transform the data
    RAF_FER_unsup_map_projection = umap_model.fit_transform(x_train)
    
    # Store the projection
    RAF_FER_unsup_umap_projections_10_01.append(RAF_FER_unsup_map_projection)

# Convert the list of projections to a numpy array
RAF_FER_unsup_umap_projections_10_01 = np.array(RAF_FER_unsup_umap_projections_10_01)

# Calculate mean and standard deviation of projections across runs
RAF_FER_mean_unsup_umap_projection_10_01 = np.mean(RAF_FER_unsup_umap_projections_10_01, axis=0)
RAF_FER_std_unsup_umap_projection_10_01 = np.std(RAF_FER_unsup_umap_projections_10_01, axis=0)

# Save the projections, mean, and standard deviation
np.save('RAF_FER_unsup_umap_projections_10_01.npy', RAF_FER_unsup_umap_projections_10_01)
np.save('RAF_FER_mean_unsup_projection_10_01_35.npy', RAF_FER_mean_unsup_umap_projection_10_01)
np.save('RAF_FER_std_unsup_projection_10_01_35.npy', RAF_FER_std_unsup_umap_projection_10_01)

# Output confirmation
print("UMAP projections, mean, and standard deviation have been saved with identifiers '_10_01'.")

In [ ]:
# Define mask for FER2013 and RAF
mask_fer = (y_train == 0)
mask_raf = (y_train == 1)

plt.figure(figsize=(10, 8))

# Plot FER2013
plt.scatter(
    RAF_FER_mean_unsup_umap_projection_10_01[mask_fer, 0],
    RAF_FER_mean_unsup_umap_projection_10_01[mask_fer, 1],
    color='blue', label="FER2013", marker='o', s=10, alpha=0.7
)

# Plot RAF
plt.scatter(
    RAF_FER_mean_unsup_umap_projection_10_01[mask_raf, 0],
    RAF_FER_mean_unsup_umap_projection_10_01[mask_raf, 1],
    color='red', label="RAF", marker='^', s=10, alpha=0.7
)

plt.title("Unsup. UMAP Mean Projection: Dataset Identification (RAF vs FER2013)")
plt.xlabel("UMAP Component 1")
plt.ylabel("UMAP Component 2")
plt.legend()
plt.show()

In [ ]:
# Calculate Silhouette Score
RAF_FER_unsup_sil_score = silhouette_score(RAF_FER_mean_unsup_umap_projection_10_01, y_train)
print(f"Silhouette Score: {RAF_FER_unsup_sil_score:.2f}")
# Calculate Davies-Bouldin Index
RAF_FER_unsup_db_score = davies_bouldin_score(RAF_FER_mean_unsup_umap_projection_10_01, y_train)
print(f"Davies-Bouldin Index: {RAF_FER_unsup_db_score:.2f}")

In [ ]:
# Apply KMeans to the UMAP-reduced data
n_clusters = len(np.unique(y_train))  # Number of clusters = number of unique labels
kmeans = KMeans(n_clusters=n_clusters, random_state=42)
RAF_FER_mean_unsup_predicted_labels = kmeans.fit_predict(RAF_FER_mean_unsup_umap_projection_10_01)

ari = adjusted_rand_score(y_train, RAF_FER_mean_unsup_predicted_labels)
print(f"Adjusted Rand Index (ARI): {ari:.2f}")

nmi = normalized_mutual_info_score(y_train, RAF_FER_mean_unsup_predicted_labels)
print(f"Normalized Mutual Information (NMI): {nmi:.2f}")

In [ ]:
# Visualize the UMAP results for 1 run
plt.figure(figsize=(10, 8))
plt.scatter(x_train_umap2[:, 0], x_train_umap2[:, 1], c=y_train, cmap="coolwarm", s=5, alpha=0.8)
plt.title("UMAP Projection: Dataset Identification (RAF vs FER2013)")
plt.xlabel("UMAP Component 1")
plt.ylabel("UMAP Component 2")
plt.colorbar(label="Dataset (0=FER2013, 1=RAF)")

In [ ]:
from sklearn.metrics import silhouette_score

In [ ]:
# Calculate silhouette score
sil_score = silhouette_score(x_train_umap, y_train)
sil_score2 = silhouette_score(x_train_umap2, y_train)
print("Silhouette Score:", sil_score, sil_score2)

**PCA + UMAP**	Faster; simple to implement; removes noise.	May lose critical image features.

**ResNet50 + UMAP** Leverages pretrained features for better results.	Slower; requires more memory.

----------

## UMAP FER2013

#### Supervised UMAP Standalone

In [36]:
# load the projections, mean, and standard deviation for the training set
fer_sup_umap_projections_train_10_01= np.load('fer_sup_umap_projections_train_10_01.npy')
fer_mean_sup_umap_projection_train_10_01= np.load('fer_mean_sup_umap_projection_train_10_01.npy')
fer_std_sup_umap_projection_train_10_01= np.load('fer_std_sup_umap_projection_train_10_01.npy')

# load the projections, mean, and standard deviation for the test set
fer_sup_umap_projections_test_10_01= np.load('fer_sup_umap_projections_test_10_01.npy')
fer_mean_sup_umap_projection_test_10_01= np.load('fer_mean_sup_umap_projection_test_10_01.npy')
fer_std_sup_umap_projection_test_10_01= np.load('fer_std_sup_umap_projection_test_10_01.npy')

In [ ]:
# Define parameters
n_neighbors = 10
min_dist = 0.1
n_components = 2
n_runs = 10  # Number of runs

# Store UMAP projections for training and test sets
fer_sup_umap_projections_train_10_01 = []
fer_sup_umap_projections_test_10_01 = []

# Run UMAP multiple times
for run in range(n_runs):
    print(f"Running Supervised UMAP - Iteration {run + 1}/{n_runs}...")

    # Create UMAP model
    umap_model = umap.UMAP(
        n_neighbors=n_neighbors, 
        min_dist=min_dist, 
        n_components=n_components, 
        random_state=run
    )

    # Fit and transform the training data with labels
    projection_train = umap_model.fit_transform(x_train_fer2013, y_train_fer2013)
    fer_sup_umap_projections_train_10_01.append(projection_train)
    
    # Transform the test data using the fitted model
    print(f"Running UMAP on Test Set - Iteration {run + 1}/{n_runs}...")
    projection_test = umap_model.transform(x_test_fer2013)
    fer_sup_umap_projections_test_10_01.append(projection_test)

# Convert the list of projections to numpy arrays
fer_sup_umap_projections_train_10_01 = np.array(fer_sup_umap_projections_train_10_01)
fer_sup_umap_projections_test_10_01 = np.array(fer_sup_umap_projections_test_10_01)

# Calculate mean and standard deviation of projections across runs (train and test)
fer_mean_sup_umap_projection_train_10_01 = np.mean(fer_sup_umap_projections_train_10_01, axis=0)
fer_std_sup_umap_projection_train_10_01 = np.std(fer_sup_umap_projections_train_10_01, axis=0)

fer_mean_sup_umap_projection_test_10_01 = np.mean(fer_sup_umap_projections_test_10_01, axis=0)
fer_std_sup_umap_projection_test_10_01 = np.std(fer_sup_umap_projections_test_10_01, axis=0)

# Save the projections, mean, and standard deviation for the training set
np.save('fer_sup_umap_projections_train_10_01.npy', fer_sup_umap_projections_train_10_01)
np.save('fer_mean_sup_umap_projection_train_10_01.npy', fer_mean_sup_umap_projection_train_10_01)
np.save('fer_std_sup_umap_projection_train_10_01.npy', fer_std_sup_umap_projection_train_10_01)

# Save the projections, mean, and standard deviation for the test set
np.save('fer_sup_umap_projections_test_10_01.npy', fer_sup_umap_projections_test_10_01)
np.save('fer_mean_sup_umap_projection_test_10_01.npy', fer_mean_sup_umap_projection_test_10_01)
np.save('fer_std_sup_umap_projection_test_10_01.npy', fer_std_sup_umap_projection_test_10_01)

# Output confirmation
print("Supervised UMAP projections for training and test sets, mean, and standard deviations have been saved.")

In [ ]:
# Load the projections, mean, and standard deviation for the test set
fer_sup_umap_projections_train_10_01= np.load('fer_sup_umap_projections_train_10_01.npy')
fer_mean_sup_umap_projection_train_10_01= np.load('fer_mean_sup_umap_projection_train_10_01.npy')
fer_std_sup_umap_projection_train_10_01= np.load('fer_std_sup_umap_projection_train_10_01.npy')

# Load the projections, mean, and standard deviation for the test set
fer_sup_umap_projections_test_10_01= np.load('fer_sup_umap_projections_test_10_01.npy')
fer_mean_sup_umap_projection_test_10_01= np.load('fer_mean_sup_umap_projection_test_10_01.npy')
fer_std_sup_umap_projection_test_10_01= np.load('fer_std_sup_umap_projection_test_10_01.npy')

In [ ]:
# Adjust colormap to have exactly 7 colors
unique_labels = np.unique(y_train_fer2013)
cmap = plt.cm.get_cmap("tab10", len(unique_labels))

# Create the scatter plot
plt.figure(figsize=(10, 8))
scatter = plt.scatter(fer_mean_sup_umap_projection_train_10_01[:, 0], fer_mean_sup_umap_projection_train_10_01[:, 1], c=y_train_fer2013, cmap=cmap, s=5, alpha=0.8)

# Add title and labels
plt.title("Supervised UMAP Projection of FER2013 Training Data (10 Runs)")
plt.xlabel("UMAP Component 1")
plt.ylabel("UMAP Component 2")

# Add and configure the colorbar
cbar = plt.colorbar(scatter)
cbar.set_ticks(range(0, 7))  # Ensure ticks align with labels
cbar.set_ticklabels([f"Emotion {label}" for label in range(0, 7)])  # Customize labels
cbar.set_label("Emotion Labels")

plt.show()

In [ ]:
# ARI
ari_fer_umap_sup_10_01 = adjusted_rand_score(y_test_fer2013, KNeighborsClassifier(n_neighbors=1).fit(fer_mean_sup_umap_projection_train_10_01, y_train_fer2013).predict(fer_mean_sup_umap_projection_test_10_01)) # second argument is y_test_pred_pca
print(f"ARI: {ari_fer_umap_sup_10_01:.4f}")
# Silhouette Score
silhouette_fer_umap_sup_10_01 = silhouette_score(fer_mean_sup_umap_projection_test_10_01, KNeighborsClassifier(n_neighbors=1).fit(fer_mean_sup_umap_projection_train_10_01, y_train_fer2013).predict(fer_mean_sup_umap_projection_test_10_01))
print(f"Silhouette Score: {silhouette_fer_umap_sup_10_01:.2f}")
# Use KMeans for clustering
n_clusters = len(np.unique(y_train_fer2013))  # Number of clusters = number of unique labels
kmeans = KMeans(n_clusters=n_clusters, random_state=42)

# Fit KMeans on the training set UMAP projections
predicted_labels = kmeans.fit_predict(fer_mean_sup_umap_projection_train_10_01)

# Compute Davies-Bouldin Index
fer_sup_umap_projection_10_01_db_score = davies_bouldin_score(
    fer_mean_sup_umap_projection_train_10_01,  # Training set mean projections
    predicted_labels  # Labels from KMeans clustering
)

print(f"Davies-Bouldin Index: {fer_sup_umap_projection_10_01_db_score:.2f}")

In [ ]:
# # Calculate Silhouette Score
# FER_sup_sil_score = silhouette_score(fer_mean_sup_umap_projection_train_10_01, y_train_fer2013)
# print(f"Silhouette Score: {FER_sup_sil_score:.2f}")
# # Calculate Davies-Bouldin Index
# FER_sup_db_score = davies_bouldin_score(fer_mean_sup_umap_projection_train_10_01, y_train_fer2013)
# print(f"Davies-Bouldin Index: {FER_sup_db_score:.2f}")

In [ ]:
# Step 1: Load the saved mean UMAP projections
mean_projection = np.load('mean_sup_projection_10_01_35.npy')  # Shape: (n_samples, 2)

# Step 2: Separate the mean projection by class
classes = np.unique(y_train_fer2013)
class_gaussians = {}

# Calculate the mean and covariance for each class
for c in classes:
    class_points = mean_projection[y_train_fer2013 == c]  # Filter by class
    mean = np.mean(class_points, axis=0)
    cov = np.cov(class_points, rowvar=False)
    class_gaussians[c] = {"mean": mean, "cov": cov}

# Step 3: Visualize Gaussian distributions
plt.figure(figsize=(10, 8))

# Plot UMAP embeddings for each class
for c in classes:
    class_points = mean_projection[y_train_fer2013 == c]
    plt.scatter(class_points[:, 0], class_points[:, 1], label=f"Class {c}", alpha=0.5, s=10)

    # Plot Gaussian contours
    mean = class_gaussians[c]["mean"]
    cov = class_gaussians[c]["cov"]
    x, y = np.meshgrid(
        np.linspace(mean[0] - 3, mean[0] + 3, 100), 
        np.linspace(mean[1] - 3, mean[1] + 3, 100)
    )
    pos = np.dstack((x, y))
    rv = multivariate_normal(mean, cov)
    plt.contour(x, y, rv.pdf(pos), levels=5, alpha=0.8)

plt.title("UMAP Mean Projections with Gaussian Distributions per Class")
plt.xlabel("UMAP Component 1")
plt.ylabel("UMAP Component 2")
plt.legend()
plt.show()

# Step 4: Evaluate likelihood for a random point
random_point = np.array([0, 0])  # Example point in UMAP space
likelihoods = {c: multivariate_normal(class_gaussians[c]["mean"], class_gaussians[c]["cov"]).pdf(random_point)
               for c in classes}

print("Likelihoods for Random Point:", likelihoods)


A very low likelihood value (e.g., 1.997e-18 or 2.528e-110) means the random point is far from the mean and not likely to belong to that class.
A higher likelihood value (e.g., 7.198e-08) indicates that the point is relatively closer to the Gaussian distribution center for that class.
Class 1 (7.198e-08) and Class 5 (1.722e-09) show higher likelihoods compared to the others.
This suggests that the random point is closer to the Gaussian centers of these classes.
For Classes 3, 4, and 6, the likelihood values are extremely small (e-110, e-29, e-67), meaning the point is far away from those Gaussian distributions.

In [ ]:
log_likelihoods = {c: multivariate_normal.logpdf(random_point, mean=class_gaussians[c]["mean"], cov=class_gaussians[c]["cov"])
                   for c in classes}

print("Log-Likelihoods for Random Point:", log_likelihoods)
# Closer to 0 means higher likelihood.
# Larger negative values mean the point is further away from the Gaussian center.

#### Unsupervised UMAP Standalone

In [38]:
# load the projections, mean, and standard deviation for the training set
fer_unsup_umap_projections_train_10_01= np.load('fer_unsup_umap_projections_train_10_01.npy')
fer_mean_unsup_umap_projection_train_10_01= np.load('fer_mean_unsup_umap_projection_train_10_01.npy')
fer_std_unsup_umap_projection_train_10_01= np.load('fer_std_unsup_umap_projection_train_10_01.npy')

# load the projections, mean, and standard deviation for the test set
fer_unsup_umap_projections_test_10_01= np.load('fer_unsup_umap_projections_test_10_01.npy')
fer_mean_unsup_umap_projection_test_10_01= np.load('fer_mean_unsup_umap_projection_test_10_01.npy')
fer_std_unsup_umap_projection_test_10_01= np.load('fer_std_unsup_umap_projection_test_10_01.npy')

In [ ]:
# Define parameters
n_neighbors = 10
min_dist = 0.1
n_components = 2
n_runs = 10  # Number of runs

# Store UMAP projections for each run (train and test)
fer_unsup_umap_projections_train_10_01 = []
fer_unsup_umap_projections_test_10_01 = []

# Run UMAP multiple times for the training set
for run in range(n_runs):
    print(f"Running UMAP on Training Set - Iteration {run + 1}/{n_runs}...")
    # Create UMAP model
    umap_model = umap.UMAP(n_neighbors=n_neighbors, min_dist=min_dist, n_components=n_components, random_state=run)
    
    # Fit and transform the training data
    projection_train = umap_model.fit_transform(x_train_fer2013)
    fer_unsup_umap_projections_train_10_01.append(projection_train)
    
    # Transform the test data using the same fitted model
    print(f"Running UMAP on Test Set - Iteration {run + 1}/{n_runs}...")
    projection_test = umap_model.transform(x_test_fer2013)
    fer_unsup_umap_projections_test_10_01.append(projection_test)

# Convert the list of projections to numpy arrays
fer_unsup_umap_projections_train_10_01 = np.array(fer_unsup_umap_projections_train_10_01)
fer_unsup_umap_projections_test_10_01 = np.array(fer_unsup_umap_projections_test_10_01)

# Calculate mean and standard deviation of projections across runs (train and test)
fer_mean_unsup_umap_projection_train_10_01 = np.mean(fer_unsup_umap_projections_train_10_01, axis=0)
fer_std_unsup_umap_projection_train_10_01 = np.std(fer_unsup_umap_projections_train_10_01, axis=0)

fer_mean_unsup_umap_projection_test_10_01 = np.mean(fer_unsup_umap_projections_test_10_01, axis=0)
fer_std_unsup_umap_projection_test_10_01 = np.std(fer_unsup_umap_projections_test_10_01, axis=0)

# Save the projections, mean, and standard deviation for the training set
np.save('fer_unsup_umap_projections_train_10_01.npy', fer_unsup_umap_projections_train_10_01)
np.save('fer_mean_unsup_umap_projection_train_10_01.npy', fer_mean_unsup_umap_projection_train_10_01)
np.save('fer_std_unsup_umap_projection_train_10_01.npy', fer_std_unsup_umap_projection_train_10_01)

# Save the projections, mean, and standard deviation for the test set
np.save('fer_unsup_umap_projections_test_10_01.npy', fer_unsup_umap_projections_test_10_01)
np.save('fer_mean_unsup_umap_projection_test_10_01.npy', fer_mean_unsup_umap_projection_test_10_01)
np.save('fer_std_unsup_umap_projection_test_10_01.npy', fer_std_unsup_umap_projection_test_10_01)

# Output confirmation
print("UMAP projections for training and test sets, mean, and standard deviations have been saved.")

In [ ]:
# Adjust colormap to have exactly 7 colors
unique_labels = np.unique(y_train_fer2013)
cmap = plt.cm.get_cmap("tab10", len(unique_labels))

# Create the scatter plot
plt.figure(figsize=(10, 8))
scatter = plt.scatter(
    fer_mean_unsup_umap_projection_train_10_01[:, 0],
    fer_mean_unsup_umap_projection_train_10_01[:, 1],
    c=y_train_fer2013, cmap=cmap, s=5, alpha=0.8
)

# Add title and labels
plt.title("Unsupervised UMAP Projection of FER2013 Training Data (10 Runs)")
plt.xlabel("UMAP Component 1")
plt.ylabel("UMAP Component 2")

# Add and configure the colorbar
cbar = plt.colorbar(scatter)
cbar.set_ticks(range(0, 7))  # Ensure ticks align with labels
cbar.set_ticklabels([f"Emotion {label}" for label in range(0, 7)])  # Customize labels
cbar.set_label("Emotion Labels")

plt.show()

In [ ]:
# ARI
ari_fer_umap_unsup_10_01 = adjusted_rand_score(y_test_fer2013, KNeighborsClassifier(n_neighbors=1).fit(fer_mean_unsup_umap_projection_train_10_01, y_train_fer2013).predict(fer_mean_unsup_umap_projection_test_10_01)) # second argument is y_test_pred_pca
print(f"ARI: {ari_fer_umap_unsup_10_01:.4f}")
# Silhouette Score
silhouette_fer_umap_unsup_10_01 = silhouette_score(fer_mean_unsup_umap_projection_test_10_01, KNeighborsClassifier(n_neighbors=1).fit(fer_mean_unsup_umap_projection_train_10_01, y_train_fer2013).predict(fer_mean_unsup_umap_projection_test_10_01))
print(f"Silhouette Score: {silhouette_fer_umap_unsup_10_01:.2f}")
# Use KMeans for clustering
n_clusters = len(np.unique(y_train_fer2013))  # Number of clusters = number of unique labels
kmeans = KMeans(n_clusters=n_clusters, random_state=42)

# Fit KMeans on the training set UMAP projections
predicted_labels = kmeans.fit_predict(fer_mean_unsup_umap_projection_train_10_01)

# Compute Davies-Bouldin Index
fer_unsup_umap_projection_10_01_db_score = davies_bouldin_score(
    fer_mean_unsup_umap_projection_train_10_01,  # Training set mean projections
    predicted_labels  # Labels from KMeans clustering
)

print(f"Davies-Bouldin Index: {fer_unsup_umap_projection_10_01_db_score:.2f}")

n_neighbors =50

In [ ]:
# Define parameters
n_neighbors = 50
min_dist = 0.1
n_components = 2
n_runs = 10  # Number of runs

# Store UMAP projections for each run (train and test)
fer_unsup_umap_projections_train_50_01 = []
fer_unsup_umap_projections_test_50_01 = []

# Run UMAP multiple times for the training set
for run in range(n_runs):
    print(f"Running UMAP on Training Set - Iteration {run + 1}/{n_runs}...")
    # Create UMAP model
    umap_model = umap.UMAP(n_neighbors=n_neighbors, min_dist=min_dist, n_components=n_components, random_state=run)
    
    # Fit and transform the training data
    projection_train = umap_model.fit_transform(x_train_fer2013)
    fer_unsup_umap_projections_train_50_01.append(projection_train)
    
    # Transform the test data using the same fitted model
    print(f"Running UMAP on Test Set - Iteration {run + 1}/{n_runs}...")
    projection_test = umap_model.transform(x_test_fer2013)
    fer_unsup_umap_projections_test_50_01.append(projection_test)

# Convert the list of projections to numpy arrays
fer_unsup_umap_projections_train_50_01 = np.array(fer_unsup_umap_projections_train_50_01)
fer_unsup_umap_projections_test_50_01 = np.array(fer_unsup_umap_projections_test_50_01)

# Calculate mean and standard deviation of projections across runs (train and test)
fer_mean_unsup_umap_projection_train_50_01 = np.mean(fer_unsup_umap_projections_train_50_01, axis=0)
fer_std_unsup_umap_projection_train_50_01 = np.std(fer_unsup_umap_projections_train_50_01, axis=0)

fer_mean_unsup_umap_projection_test_50_01 = np.mean(fer_unsup_umap_projections_test_50_01, axis=0)
fer_std_unsup_umap_projection_test_50_01 = np.std(fer_unsup_umap_projections_test_50_01, axis=0)

# Save the projections, mean, and standard deviation for the training set
np.save('fer_unsup_umap_projections_train_50_01.npy', fer_unsup_umap_projections_train_50_01)
np.save('fer_mean_unsup_umap_projection_train_50_01.npy', fer_mean_unsup_umap_projection_train_50_01)
np.save('fer_std_unsup_umap_projection_train_50_01.npy', fer_std_unsup_umap_projection_train_50_01)

# Save the projections, mean, and standard deviation for the test set
np.save('fer_unsup_umap_projections_test_50_01.npy', fer_unsup_umap_projections_test_50_01)
np.save('fer_mean_unsup_umap_projection_test_50_01.npy', fer_mean_unsup_umap_projection_test_50_01)
np.save('fer_std_unsup_umap_projection_test_50_01.npy', fer_std_unsup_umap_projection_test_50_01)

# Output confirmation
print("UMAP projections for training and test sets, mean, and standard deviations have been saved.")

In [ ]:
# Adjust colormap to have exactly 7 colors
unique_labels = np.unique(y_train_fer2013)
cmap = plt.cm.get_cmap("tab10", len(unique_labels))

# Create the scatter plot
plt.figure(figsize=(10, 8))
scatter = plt.scatter(
    fer_mean_unsup_umap_projection_train_50_01[:, 0],
    fer_mean_unsup_umap_projection_train_50_01[:, 1],
    c=y_train_fer2013, cmap=cmap, s=5, alpha=0.8
)

# Add title and labels
plt.title("n_neighbours=50 Unsupervised UMAP Projection of FER2013 Training Data (10 Runs)")
plt.xlabel("UMAP Component 1")
plt.ylabel("UMAP Component 2")

# Add and configure the colorbar
cbar = plt.colorbar(scatter)
cbar.set_ticks(range(0, 7))  # Ensure ticks align with labels
cbar.set_ticklabels([f"Emotion {label}" for label in range(0, 7)])  # Customize labels
cbar.set_label("Emotion Labels")

plt.show()

In [ ]:
# ARI
ari_fer_umap_unsup_50_01 = adjusted_rand_score(y_test_fer2013, KNeighborsClassifier(n_neighbors=1).fit(fer_mean_unsup_umap_projection_train_50_01, y_train_fer2013).predict(fer_mean_unsup_umap_projection_test_50_01)) # second argument is y_test_pred_pca
print(f"ARI: {ari_fer_umap_unsup_50_01:.4f}")
# Silhouette Score
silhouette_fer_umap_unsup_50_01 = silhouette_score(fer_mean_unsup_umap_projection_test_50_01, KNeighborsClassifier(n_neighbors=1).fit(fer_mean_unsup_umap_projection_train_50_01, y_train_fer2013).predict(fer_mean_unsup_umap_projection_test_50_01))
print(f"Silhouette Score: {silhouette_fer_umap_unsup_50_01:.2f}")
# Use KMeans for clustering
n_clusters = len(np.unique(y_train_fer2013))  # Number of clusters = number of unique labels
kmeans = KMeans(n_clusters=n_clusters, random_state=42)

# Fit KMeans on the training set UMAP projections
predicted_labels = kmeans.fit_predict(fer_mean_unsup_umap_projection_train_50_01)

# Compute Davies-Bouldin Index
fer_unsup_umap_projection_50_01_db_score = davies_bouldin_score(
    fer_mean_unsup_umap_projection_train_50_01,  # Training set mean projections
    predicted_labels  # Labels from KMeans clustering
)

print(f"Davies-Bouldin Index: {fer_unsup_umap_projection_50_01_db_score:.2f}")

In [ ]:
# # Step 1: Load the saved mean UMAP projections
# fer_mean_projection = np.load('fer_mean_unsup_umap_projection_train_10_01.npy')  # Shape: (n_samples, 2)

# # Step 2: Separate the mean projection by class
# fer_classes = np.unique(y_train_fer2013)
# fer_class_gaussians = {}

# # Calculate the mean and covariance for each class
# for c in fer_classes:
#     class_points = fer_mean_projection[y_train_fer2013 == c]  # Filter by class
#     mean = np.mean(class_points, axis=0)
#     cov = np.cov(class_points, rowvar=False)
#     fer_class_gaussians[c] = {"mean": mean, "cov": cov}

# # Step 3: Visualize Gaussian distributions
# plt.figure(figsize=(10, 8))

# # Plot UMAP embeddings for each class
# for c in fer_classes:
#     class_points = fer_mean_projection[y_train_fer2013 == c]
#     plt.scatter(class_points[:, 0], class_points[:, 1], label=f"Class {c}", alpha=0.5, s=10)

#     # Plot Gaussian contours
#     mean = fer_class_gaussians[c]["mean"]
#     cov = fer_class_gaussians[c]["cov"]
#     x, y = np.meshgrid(
#         np.linspace(mean[0] - 3, mean[0] + 3, 100), 
#         np.linspace(mean[1] - 3, mean[1] + 3, 100)
#     )
#     pos = np.dstack((x, y))
#     rv = multivariate_normal(mean, cov)
#     plt.contour(x, y, rv.pdf(pos), levels=5, alpha=0.8)

# plt.title("UMAP Mean Projections with Gaussian Distributions per Class FER2013")
# plt.xlabel("UMAP Component 1")
# plt.ylabel("UMAP Component 2")
# plt.legend()
# plt.show()

# # Step 4: Evaluate likelihood for a random point
# random_point = np.array([0, 0])  # Example point in UMAP space
# likelihoods = {c: multivariate_normal(fer_class_gaussians[c]["mean"], fer_class_gaussians[c]["cov"]).pdf(random_point)
#                for c in fer_classes}

# print("Likelihoods for Random Point:", likelihoods)

A very low likelihood value (e.g., 1.997e-18 or 2.528e-110) means the random point is far from the mean and not likely to belong to that class.
A higher likelihood value (e.g., 7.198e-08) indicates that the point is relatively closer to the Gaussian distribution center for that class.
Class 1 (7.198e-08) and Class 5 (1.722e-09) show higher likelihoods compared to the others.
This suggests that the random point is closer to the Gaussian centers of these classes.
For Classes 3, 4, and 6, the likelihood values are extremely small (e-110, e-29, e-67), meaning the point is far away from those Gaussian distributions.

In [ ]:
log_likelihoods = {c: multivariate_normal.logpdf(random_point, mean=class_gaussians[c]["mean"], cov=class_gaussians[c]["cov"])
                   for c in classes}

print("Log-Likelihoods for Random Point:", log_likelihoods)
# Closer to 0 means higher likelihood.
# Larger negative values mean the point is further away from the Gaussian center.

-----

## PCA + UMAP

In [ ]:
# Step 2: Apply PCA
pca = PCA(0.95)
x_train_fer2013_pca_emotions = pca.fit_transform(x_train_fer2013)
x_test_fer2013_pca_emotions = pca.transform(x_test_fer2013)

print(f"Original number of features: {x_train_fer2013.shape[1]}")
print(f"Reduced number of features: {x_train_fer2013_pca_emotions.shape[1]}")

In [55]:
# Save the projections, mean, and standard deviation
np.save('x_train_fer2013_pca_emotions.npy', x_train_fer2013_pca_emotions)
np.save('x_test_fer2013_pca_emotions.npy', x_test_fer2013_pca_emotions)

### PCA + UMAP Unsupervised 10 runs

In [40]:
# load the projections, mean, and standard deviation for the training set
fer_unsup_pca_umap_projections_train_10_01= np.load('fer_unsup_pca_umap_projections_train_10_01.npy')
fer_mean_unsup_pca_umap_projection_train_10_01= np.load('fer_mean_unsup_pca_umap_projection_train_10_01.npy')
fer_std_unsup_pca_umap_projection_train_10_01= np.load('fer_std_unsup_pca_umap_projection_train_10_01.npy')

# load the projections, mean, and standard deviation for the test set
fer_unsup_pca_umap_projections_test_10_01= np.load('fer_unsup_pca_umap_projections_test_10_01.npy')
fer_mean_unsup_pca_umap_projection_test_10_01= np.load('fer_mean_unsup_pca_umap_projection_test_10_01.npy')
fer_std_unsup_pca_umap_projection_test_10_01= np.load('fer_std_unsup_pca_umap_projection_test_10_01.npy')

In [ ]:
# Define parameters
n_neighbors = 10
min_dist = 0.1
n_components = 2
n_runs = 10  # Number of runs

# Store UMAP projections for each run (train and test)
fer_unsup_pca_umap_projections_train_10_01 = []
fer_unsup_pca_umap_projections_test_10_01 = []

# Run UMAP multiple times for the training set
for run in range(n_runs):
    print(f"Running UMAP on Training Set - Iteration {run + 1}/{n_runs}...")
    # Create UMAP model
    umap_model = umap.UMAP(n_neighbors=n_neighbors, min_dist=min_dist, n_components=n_components, random_state=run)
    
    # Fit and transform the training data
    projection_train = umap_model.fit_transform(x_train_fer2013_pca_emotions)
    fer_unsup_pca_umap_projections_train_10_01.append(projection_train)
    
    # Transform the test data using the same fitted model
    print(f"Running UMAP on Test Set - Iteration {run + 1}/{n_runs}...")
    projection_test = umap_model.transform(x_test_fer2013_pca_emotions)
    fer_unsup_pca_umap_projections_test_10_01.append(projection_test)

# Convert the list of projections to numpy arrays
fer_unsup_pca_umap_projections_train_10_01 = np.array(fer_unsup_pca_umap_projections_train_10_01)
fer_unsup_pca_umap_projections_test_10_01 = np.array(fer_unsup_pca_umap_projections_test_10_01)

# Calculate mean and standard deviation of projections across runs (train and test)
fer_mean_unsup_pca_umap_projection_train_10_01 = np.mean(fer_unsup_pca_umap_projections_train_10_01, axis=0)
fer_std_unsup_pca_umap_projection_train_10_01 = np.std(fer_unsup_pca_umap_projections_train_10_01, axis=0)

fer_mean_unsup_pca_umap_projection_test_10_01 = np.mean(fer_unsup_pca_umap_projections_test_10_01, axis=0)
fer_std_unsup_pca_umap_projection_test_10_01 = np.std(fer_unsup_pca_umap_projections_test_10_01, axis=0)

# Save the projections, mean, and standard deviation for the training set
np.save('fer_unsup_pca_umap_projections_train_10_01.npy', fer_unsup_pca_umap_projections_train_10_01)
np.save('fer_mean_unsup_pca_umap_projection_train_10_01.npy', fer_mean_unsup_pca_umap_projection_train_10_01)
np.save('fer_std_unsup_pca_umap_projection_train_10_01.npy', fer_std_unsup_pca_umap_projection_train_10_01)

# Save the projections, mean, and standard deviation for the test set
np.save('fer_unsup_pca_umap_projections_test_10_01.npy', fer_unsup_pca_umap_projections_test_10_01)
np.save('fer_mean_unsup_pca_umap_projection_test_10_01.npy', fer_mean_unsup_pca_umap_projection_test_10_01)
np.save('fer_std_unsup_pca_umap_projection_test_10_01.npy', fer_std_unsup_pca_umap_projection_test_10_01)

# Output confirmation
print("UMAP projections for training and test sets, mean, and standard deviations have been saved.")

In [ ]:
# Adjust colormap to have exactly 7 colors
unique_labels = np.unique(y_train_fer2013)
cmap = plt.cm.get_cmap("tab10", len(unique_labels))

# Create the scatter plot
plt.figure(figsize=(10, 8))
scatter = plt.scatter(
    fer_mean_unsup_pca_umap_projection_train_10_01[:, 0],
    fer_mean_unsup_pca_umap_projection_train_10_01[:, 1],
    c=y_train_fer2013, cmap=cmap, s=5, alpha=0.8
)

# Add title and labels
plt.title("PCA + Unsupervised UMAP Projection of FER2013 Training Data (10 Runs)")
plt.xlabel("UMAP Component 1")
plt.ylabel("UMAP Component 2")

# Add and configure the colorbar
cbar = plt.colorbar(scatter)
cbar.set_ticks(range(0, 7))  # Ensure ticks align with labels
cbar.set_ticklabels([f"Emotion {label}" for label in range(0, 7)])  # Customize labels
cbar.set_label("Emotion Labels")

plt.show()

In [ ]:
# ARI
ari_fer_pca_umap_unsup_10_01 = adjusted_rand_score(y_test_fer2013, KNeighborsClassifier(n_neighbors=1).fit(fer_mean_unsup_pca_umap_projection_train_10_01, y_train_fer2013).predict(fer_mean_unsup_pca_umap_projection_test_10_01))
print(f"ARI: {ari_fer_pca_umap_unsup_10_01:.4f}")
# Silhouette Score
silhouette_fer_pca_umap_unsup_10_01 = silhouette_score(fer_mean_unsup_pca_umap_projection_test_10_01, KNeighborsClassifier(n_neighbors=1).fit(fer_mean_unsup_pca_umap_projection_train_10_01, y_train_fer2013).predict(fer_mean_unsup_pca_umap_projection_test_10_01))
print(f"Silhouette Score: {silhouette_fer_pca_umap_unsup_10_01:.2f}")

# Fit KMeans on the training set UMAP projections
predicted_labels = kmeans.fit_predict(fer_mean_unsup_pca_umap_projection_train_10_01)

# Compute Davies-Bouldin Index
fer_unsup_pca_projection_10_01_db_score = davies_bouldin_score(
    fer_mean_unsup_pca_umap_projection_train_10_01,  # Training set mean projections
    predicted_labels  # Labels from KMeans clustering
)

print(f"Davies-Bouldin Index: {fer_unsup_pca_projection_10_01_db_score:.2f}")

----------

### PCA + UMAP Supervised 10 runs

In [42]:
# load the projections, mean, and standard deviation for the training set
fer_sup_pca_umap_projections_train_10_01= np.load('fer_sup_pca_umap_projections_train_10_01.npy')
fer_mean_sup_pca_umap_projection_train_10_01= np.load('fer_mean_sup_pca_umap_projection_train_10_01.npy')
fer_std_sup_pca_umap_projection_train_10_01= np.load('fer_std_sup_pca_umap_projection_train_10_01.npy')

# load the projections, mean, and standard deviation for the test set
fer_sup_pca_umap_projections_test_10_01= np.load('fer_sup_pca_umap_projections_test_10_01.npy')
fer_mean_sup_pca_umap_projection_test_10_01= np.load('fer_mean_sup_pca_umap_projection_test_10_01.npy')
fer_std_sup_pca_umap_projection_test_10_01= np.load('fer_std_sup_pca_umap_projection_test_10_01.npy')

In [ ]:
# Define parameters
n_neighbors = 10
min_dist = 0.1
n_components = 2
n_runs = 10  # Number of runs

# Store UMAP projections for each run (train and test)
fer_sup_pca_umap_projections_train_10_01 = []
fer_sup_pca_umap_projections_test_10_01 = []

# Run UMAP multiple times for the training set
for run in range(n_runs):
    print(f"Running UMAP on Training Set - Iteration {run + 1}/{n_runs}...")
    # Create UMAP model
    umap_model = umap.UMAP(n_neighbors=n_neighbors, min_dist=min_dist, n_components=n_components, random_state=run)
    
    # Fit and transform the training data
    projection_train = umap_model.fit_transform(x_train_fer2013_pca_emotions,y_train_fer2013)
    fer_sup_pca_umap_projections_train_10_01.append(projection_train)
    
    # Transform the test data using the same fitted model
    print(f"Running UMAP on Test Set - Iteration {run + 1}/{n_runs}...")
    projection_test = umap_model.transform(x_test_fer2013_pca_emotions)
    fer_sup_pca_umap_projections_test_10_01.append(projection_test)

# Convert the list of projections to numpy arrays
fer_sup_pca_umap_projections_train_10_01 = np.array(fer_sup_pca_umap_projections_train_10_01)
fer_sup_pca_umap_projections_test_10_01 = np.array(fer_sup_pca_umap_projections_test_10_01)

# Calculate mean and standard deviation of projections across runs (train and test)
fer_mean_sup_pca_umap_projection_train_10_01 = np.mean(fer_sup_pca_umap_projections_train_10_01, axis=0)
fer_std_sup_pca_umap_projection_train_10_01 = np.std(fer_sup_pca_umap_projections_train_10_01, axis=0)

fer_mean_sup_pca_umap_projection_test_10_01 = np.mean(fer_sup_pca_umap_projections_test_10_01, axis=0)
fer_std_sup_pca_umap_projection_test_10_01 = np.std(fer_sup_pca_umap_projections_test_10_01, axis=0)

# Save the projections, mean, and standard deviation for the training set
np.save('fer_sup_pca_umap_projections_train_10_01.npy', fer_sup_pca_umap_projections_train_10_01)
np.save('fer_mean_sup_pca_umap_projection_train_10_01.npy', fer_mean_sup_pca_umap_projection_train_10_01)
np.save('fer_std_sup_pca_umap_projection_train_10_01.npy', fer_std_sup_pca_umap_projection_train_10_01)

# Save the projections, mean, and standard deviation for the test set
np.save('fer_sup_pca_umap_projections_test_10_01.npy', fer_sup_pca_umap_projections_test_10_01)
np.save('fer_mean_sup_pca_umap_projection_test_10_01.npy', fer_mean_sup_pca_umap_projection_test_10_01)
np.save('fer_std_sup_pca_umap_projection_test_10_01.npy', fer_std_sup_pca_umap_projection_test_10_01)

# Output confirmation
print("UMAP projections for training and test sets, mean, and standard deviations have been saved.")

In [ ]:
# Adjust colormap to have exactly 7 colors
unique_labels = np.unique(y_train_fer2013)
cmap = plt.cm.get_cmap("tab10", len(unique_labels))

# Create the scatter plot
plt.figure(figsize=(10, 8))
scatter = plt.scatter(
    fer_mean_sup_pca_umap_projection_train_10_01[:, 0],
    fer_mean_sup_pca_umap_projection_train_10_01[:, 1],
    c=y_train_fer2013, cmap=cmap, s=5, alpha=0.8
)

# Add title and labels
plt.title("PCA + Supervised UMAP Projection of FER2013 Training Data (10 Runs)")
plt.xlabel("UMAP Component 1")
plt.ylabel("UMAP Component 2")

# Add and configure the colorbar
cbar = plt.colorbar(scatter)
cbar.set_ticks(range(0, 7))  # Ensure ticks align with labels
cbar.set_ticklabels([f"Emotion {label}" for label in range(0, 7)])  # Customize labels
cbar.set_label("Emotion Labels")

plt.show()

In [ ]:
# ARI
ari_fer_pca_umap_sup_10_01 = adjusted_rand_score(y_test_fer2013, KNeighborsClassifier(n_neighbors=1).fit(fer_mean_sup_pca_umap_projection_train_10_01, y_train_fer2013).predict(fer_mean_sup_pca_umap_projection_test_10_01))
print(f"ARI: {ari_fer_pca_umap_sup_10_01:.4f}")
# Silhouette Score
silhouette_fer_pca_umap_sup_10_01 = silhouette_score(fer_mean_sup_pca_umap_projection_test_10_01, KNeighborsClassifier(n_neighbors=1).fit(fer_mean_sup_pca_umap_projection_train_10_01, y_train_fer2013).predict(fer_mean_sup_pca_umap_projection_test_10_01))
print(f"Silhouette Score: {silhouette_fer_pca_umap_sup_10_01:.2f}")

# Use KMeans for clustering
n_clusters = len(np.unique(y_train_fer2013))  # Number of clusters = number of unique labels
kmeans = KMeans(n_clusters=n_clusters, random_state=42)

# Fit KMeans on the training set UMAP projections
predicted_labels = kmeans.fit_predict(fer_mean_sup_pca_umap_projection_train_10_01)

# Compute Davies-Bouldin Index
fer_sup_umap_pca_projection_10_01_db_score = davies_bouldin_score(
    fer_mean_sup_pca_umap_projection_train_10_01,  # Training set mean projections
    predicted_labels  # Labels from KMeans clustering
)

print(f"Davies-Bouldin Index: {fer_sup_umap_pca_projection_10_01_db_score:.2f}")

---------

## Gabor + PCA + UMAP

### Gabor + PCA + Unsupervised UMAP

In [44]:
# load the projections, mean, and standard deviation for the training set
fer_unsup_gabor_pca_umap_projections_train_10_01= np.load('fer_unsup_gabor_pca_umap_projections_train_10_01.npy')
fer_mean_unsup_gabor_pca_umap_projection_train_10_01= np.load('fer_mean_unsup_gabor_pca_umap_projection_train_10_01.npy')
fer_std_unsup_gabor_pca_umap_projection_train_10_01= np.load('fer_std_unsup_gabor_pca_umap_projection_train_10_01.npy')

# load the projections, mean, and standard deviation for the test set
fer_unsup_gabor_pca_umap_projections_test_10_01= np.load('fer_unsup_gabor_pca_umap_projections_test_10_01.npy')
fer_mean_unsup_gabor_pca_umap_projection_test_10_01= np.load('fer_mean_unsup_gabor_pca_umap_projection_test_10_01.npy')
fer_std_unsup_gabor_pca_umap_projection_test_10_01= np.load('fer_std_unsup_gabor_pca_umap_projection_test_10_01.npy')

In [66]:
import cv2

In [67]:
# Create Gabor Kernels
def create_gabor_kernels():
    """Generates a set of Gabor kernels with different orientations and frequencies."""
    kernels = []
    ksize = 31  # Kernel size
    sigma = 4.0  # Standard deviation of the Gaussian envelope
    lambd = 10.0  # Wavelength of the sinusoidal factor
    gamma = 0.5  # Spatial aspect ratio
    for theta in np.arange(0, np.pi, np.pi / 4):  # 8 orientations
        kernel = cv2.getGaborKernel((ksize, ksize), sigma, theta, lambd, gamma, psi=0, ktype=cv2.CV_32F)
        kernels.append(kernel)
    return kernels

# Apply Gabor Filters
def apply_gabor_filters(images, kernels):
    """Applies a set of Gabor filters to a batch of images."""
    gabor_features = []
    for image in images:
        image_2d = image.reshape(48, 48)  # Reshape back to 2D (assumes 48x48 images)
        responses = []
        for kernel in kernels:
            filtered = cv2.filter2D(image_2d, cv2.CV_32F, kernel)  # Apply Gabor filter
            responses.append(filtered.flatten())  # Flatten the filtered image
        gabor_features.append(np.concatenate(responses))  # Concatenate all filter responses
    return np.array(gabor_features)

In [ ]:
# Generate Gabor kernels
gabor_kernels = create_gabor_kernels()
print(f"Generated {len(gabor_kernels)} Gabor kernels.")

In [ ]:
# Apply Gabor filters to the training and test sets
fer_x_train_gabor = apply_gabor_filters(x_train_fer2013, gabor_kernels)
fer_x_test_gabor = apply_gabor_filters(x_test_fer2013, gabor_kernels)

print(f"Train Gabor feature shape: {fer_x_train_gabor.shape}")
print(f"Test Gabor feature shape: {fer_x_test_gabor.shape}")

In [70]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
fer_x_train_gabor = scaler.fit_transform(fer_x_train_gabor)
fer_x_test_gabor = scaler.transform(fer_x_test_gabor)

Applying PCA to Gabor

In [ ]:
pca = PCA(0.95)
x_fer_train_gabor_pca = pca.fit_transform(fer_x_train_gabor)
x_fer_test_gabor_pca = pca.transform(fer_x_test_gabor)

print(f"Reduced train shape: {x_fer_train_gabor_pca.shape}")
print(f"Reduced test shape: {x_fer_test_gabor_pca.shape}")

In [72]:
np.save('x_raf_train_gabor_pca.npy', x_fer_train_gabor_pca)
np.save('x_raf_test_gabor_pca.npy', x_fer_test_gabor_pca)

In [ ]:
# Define parameters
n_neighbors = 10
min_dist = 0.1
n_components = 2
n_runs = 10  # Number of runs

# Store UMAP projections for each run (train and test)
fer_unsup_gabor_pca_umap_projections_train_10_01 = []
fer_unsup_gabor_pca_umap_projections_test_10_01 = []

# Run UMAP multiple times for the training set
for run in range(n_runs):
    print(f"Running UMAP on Training Set - Iteration {run + 1}/{n_runs}...")
    # Create UMAP model
    umap_model = umap.UMAP(n_neighbors=n_neighbors, min_dist=min_dist, n_components=n_components, random_state=run)
    
    # Fit and transform the training data
    projection_train = umap_model.fit_transform(x_fer_train_gabor_pca)
    fer_unsup_gabor_pca_umap_projections_train_10_01.append(projection_train)
    
    # Transform the test data using the same fitted model
    print(f"Running UMAP on Test Set - Iteration {run + 1}/{n_runs}...")
    projection_test = umap_model.transform(x_fer_test_gabor_pca)
    fer_unsup_gabor_pca_umap_projections_test_10_01.append(projection_test)

# Convert the list of projections to numpy arrays
fer_unsup_gabor_pca_umap_projections_train_10_01 = np.array(fer_unsup_gabor_pca_umap_projections_train_10_01)
fer_unsup_gabor_pca_umap_projections_test_10_01 = np.array(fer_unsup_gabor_pca_umap_projections_test_10_01)

# Calculate mean and standard deviation of projections across runs (train and test)
fer_mean_unsup_gabor_pca_umap_projection_train_10_01 = np.mean(fer_unsup_gabor_pca_umap_projections_train_10_01, axis=0)
fer_std_unsup_gabor_pca_umap_projection_train_10_01 = np.std(fer_unsup_gabor_pca_umap_projections_train_10_01, axis=0)

fer_mean_unsup_gabor_pca_umap_projection_test_10_01 = np.mean(fer_unsup_gabor_pca_umap_projections_test_10_01, axis=0)
fer_std_unsup_gabor_pca_umap_projection_test_10_01 = np.std(fer_unsup_gabor_pca_umap_projections_test_10_01, axis=0)

# Save the projections, mean, and standard deviation for the training set
np.save('fer_unsup_gabor_pca_umap_projections_train_10_01.npy', fer_unsup_gabor_pca_umap_projections_train_10_01)
np.save('fer_mean_unsup_gabor_pca_umap_projection_train_10_01.npy', fer_mean_unsup_gabor_pca_umap_projection_train_10_01)
np.save('fer_std_unsup_gabor_pca_umap_projection_train_10_01.npy', fer_std_unsup_gabor_pca_umap_projection_train_10_01)

# Save the projections, mean, and standard deviation for the test set
np.save('fer_unsup_gabor_pca_umap_projections_test_10_01.npy', fer_unsup_gabor_pca_umap_projections_test_10_01)
np.save('fer_mean_unsup_gabor_pca_umap_projection_test_10_01.npy', fer_mean_unsup_gabor_pca_umap_projection_test_10_01)
np.save('fer_std_unsup_gabor_pca_umap_projection_test_10_01.npy', fer_std_unsup_gabor_pca_umap_projection_test_10_01)

# Output confirmation
print("UMAP projections for training and test sets, mean, and standard deviations have been saved.")

In [ ]:
# Adjust colormap to have exactly 7 colors
unique_labels = np.unique(y_train_fer2013)
cmap = plt.cm.get_cmap("tab10", len(unique_labels))

# Create the scatter plot
plt.figure(figsize=(10, 8))
scatter = plt.scatter(
    fer_mean_unsup_gabor_pca_umap_projection_train_10_01[:, 0],
    fer_mean_unsup_gabor_pca_umap_projection_train_10_01[:, 1],
    c=y_train_fer2013, cmap=cmap, s=5, alpha=0.8
)

# Add title and labels
plt.title("Gabor + PCA + Unsupervised UMAP Projection of FER2013 Training Data (10 Runs)")
plt.xlabel("UMAP Component 1")
plt.ylabel("UMAP Component 2")

# Add and configure the colorbar
cbar = plt.colorbar(scatter)
cbar.set_ticks(range(0, 7))  # Ensure ticks align with labels
cbar.set_ticklabels([f"Emotion {label}" for label in range(0, 7)])  # Customize labels
cbar.set_label("Emotion Labels")

plt.show()

In [ ]:
# ARI
ari_fer_gabor_pca_umap_unsup_10_01 = adjusted_rand_score(y_test_fer2013, KNeighborsClassifier(n_neighbors=1).fit(fer_mean_unsup_gabor_pca_umap_projection_train_10_01, y_train_fer2013).predict(fer_mean_unsup_gabor_pca_umap_projection_test_10_01)) # second argument is y_test_fer2013_pred_gabor_pca
print(f"ARI: {ari_fer_gabor_pca_umap_unsup_10_01:.4f}")
# Silhouette Score
silhouette_fer_gabor_pca_umap_unsup_10_01 = silhouette_score(fer_mean_unsup_gabor_pca_umap_projection_test_10_01, KNeighborsClassifier(n_neighbors=1).fit(fer_mean_unsup_gabor_pca_umap_projection_train_10_01, y_train_fer2013).predict(fer_mean_unsup_gabor_pca_umap_projection_test_10_01))
print(f"Silhouette Score: {silhouette_fer_gabor_pca_umap_unsup_10_01:.2f}")

# Use KMeans for clustering
n_clusters = len(np.unique(y_train_fer2013))  # Number of clusters = number of unique labels
kmeans = KMeans(n_clusters=n_clusters, random_state=42)

# Fit KMeans on the training set UMAP projections
predicted_labels = kmeans.fit_predict(fer_mean_unsup_gabor_pca_umap_projection_train_10_01)

# Compute Davies-Bouldin Index
fer_unsup_gabor_pca_umap_projection_10_01_db_score = davies_bouldin_score(
    fer_mean_unsup_gabor_pca_umap_projection_train_10_01,  # Training set mean projections
    predicted_labels  # Labels from KMeans clustering
)

print(f"Davies-Bouldin Index: {fer_unsup_gabor_pca_umap_projection_10_01_db_score:.2f}")

### Gabor + PCA + UMAP Supervised 10 runs

In [46]:
# load the projections, mean, and standard deviation for the training set
fer_sup_gabor_pca_umap_projections_train_10_01= np.load('fer_sup_gabor_pca_umap_projections_train_10_01.npy')
fer_mean_sup_gabor_pca_umap_projection_train_10_01= np.load('fer_mean_sup_gabor_pca_umap_projection_train_10_01.npy')
fer_std_sup_gabor_pca_umap_projection_train_10_01= np.load('fer_std_sup_gabor_pca_umap_projection_train_10_01.npy')

# load the projections, mean, and standard deviation for the test set
fer_sup_gabor_pca_umap_projections_test_10_01= np.load('fer_sup_gabor_pca_umap_projections_test_10_01.npy')
fer_mean_sup_gabor_pca_umap_projection_test_10_01= np.load('fer_mean_sup_gabor_pca_umap_projection_test_10_01.npy')
fer_std_sup_gabor_pca_umap_projection_test_10_01= np.load('fer_std_sup_gabor_pca_umap_projection_test_10_01.npy')

In [ ]:
# Define parameters
n_neighbors = 10
min_dist = 0.1
n_components = 2
n_runs = 10  # Number of runs

# Store UMAP projections for training and test sets
fer_sup_gabor_pca_umap_projections_train_10_01 = []
fer_sup_gabor_pca_umap_projections_test_10_01 = []

# Run UMAP multiple times
for run in range(n_runs):
    print(f"Running Supervised UMAP - Iteration {run + 1}/{n_runs}...")

    # Create UMAP model
    umap_model = umap.UMAP(
        n_neighbors=n_neighbors, 
        min_dist=min_dist, 
        n_components=n_components, 
        random_state=run
    )

    # Fit and transform the training data with labels
    projection_train = umap_model.fit_transform(x_fer_train_gabor_pca, y_train_fer2013)
    fer_sup_gabor_pca_umap_projections_train_10_01.append(projection_train)
    
    # Transform the test data using the fitted model
    print(f"Running UMAP on Test Set - Iteration {run + 1}/{n_runs}...")
    projection_test = umap_model.transform(x_fer_test_gabor_pca)
    fer_sup_gabor_pca_umap_projections_test_10_01.append(projection_test)

# Convert the list of projections to numpy arrays
fer_sup_gabor_pca_umap_projections_train_10_01 = np.array(fer_sup_gabor_pca_umap_projections_train_10_01)
fer_sup_gabor_pca_umap_projections_test_10_01 = np.array(fer_sup_gabor_pca_umap_projections_test_10_01)

# Calculate mean and standard deviation of projections across runs (train and test)
fer_mean_sup_gabor_pca_umap_projection_train_10_01 = np.mean(fer_sup_gabor_pca_umap_projections_train_10_01, axis=0)
fer_std_sup_gabor_pca_umap_projection_train_10_01 = np.std(fer_sup_gabor_pca_umap_projections_train_10_01, axis=0)

fer_mean_sup_gabor_pca_umap_projection_test_10_01 = np.mean(fer_sup_gabor_pca_umap_projections_test_10_01, axis=0)
fer_std_sup_gabor_pca_umap_projection_test_10_01 = np.std(fer_sup_gabor_pca_umap_projections_test_10_01, axis=0)

# Save the projections, mean, and standard deviation for the training set
np.save('fer_sup_gabor_pca_umap_projections_train_10_01.npy', fer_sup_gabor_pca_umap_projections_train_10_01)
np.save('fer_mean_sup_gabor_pca_umap_projection_train_10_01.npy', fer_mean_sup_gabor_pca_umap_projection_train_10_01)
np.save('fer_std_sup_gabor_pca_umap_projection_train_10_01.npy', fer_std_sup_gabor_pca_umap_projection_train_10_01)

# Save the projections, mean, and standard deviation for the test set
np.save('fer_sup_gabor_pca_umap_projections_test_10_01.npy', fer_sup_gabor_pca_umap_projections_test_10_01)
np.save('fer_mean_sup_gabor_pca_umap_projection_test_10_01.npy', fer_mean_sup_gabor_pca_umap_projection_test_10_01)
np.save('fer_std_sup_gabor_pca_umap_projection_test_10_01.npy', fer_std_sup_gabor_pca_umap_projection_test_10_01)

# Output confirmation
print("Supervised UMAP projections for training and test sets, mean, and standard deviations have been saved.")

In [ ]:
# Adjust colormap to have exactly 7 colors
unique_labels = np.unique(y_train_fer2013)
cmap = plt.cm.get_cmap("tab10", len(unique_labels))

# Create the scatter plot
plt.figure(figsize=(10, 8))
scatter = plt.scatter(
    fer_mean_sup_gabor_pca_umap_projection_train_10_01[:, 0],
    fer_mean_sup_gabor_pca_umap_projection_train_10_01[:, 1],
    c=y_train_fer2013, cmap=cmap, s=5, alpha=0.8
)

# Add title and labels
plt.title("Gabor + PCA + Supervised UMAP Projection of FER2013 Training Data (10 Runs)")
plt.xlabel("UMAP Component 1")
plt.ylabel("UMAP Component 2")

# Add and configure the colorbar
cbar = plt.colorbar(scatter)
cbar.set_ticks(range(0, 7))  # Ensure ticks align with labels
cbar.set_ticklabels([f"Emotion {label}" for label in range(0, 7)])  # Customize labels
cbar.set_label("Emotion Labels")

plt.show()

In [ ]:
# ARI
ari_fer_gabor_pca_umap_sup_10_01 = adjusted_rand_score(y_test_fer2013, KNeighborsClassifier(n_neighbors=1).fit(fer_mean_sup_gabor_pca_umap_projection_train_10_01, y_train_fer2013).predict(fer_mean_sup_gabor_pca_umap_projection_test_10_01)) # second argument is y_test_fer2013_pred_gabor_pca
print(f"ARI: {ari_fer_gabor_pca_umap_sup_10_01:.4f}")
# Silhouette Score
silhouette_fer_gabor_pca_umap_sup_10_01 = silhouette_score(fer_mean_sup_gabor_pca_umap_projection_test_10_01, KNeighborsClassifier(n_neighbors=1).fit(fer_mean_sup_gabor_pca_umap_projection_train_10_01, y_train_fer2013).predict(fer_mean_sup_gabor_pca_umap_projection_test_10_01))
print(f"Silhouette Score: {silhouette_fer_gabor_pca_umap_sup_10_01:.2f}")

# Use KMeans for clustering
n_clusters = len(np.unique(y_train_fer2013))  # Number of clusters = number of unique labels
kmeans = KMeans(n_clusters=n_clusters, random_state=42)

# Fit KMeans on the training set UMAP projections
predicted_labels = kmeans.fit_predict(fer_mean_sup_gabor_pca_umap_projection_train_10_01)

# Compute Davies-Bouldin Index
fer_sup_gabor_pca_umap_projection_10_01_db_score = davies_bouldin_score(
    fer_mean_sup_gabor_pca_umap_projection_train_10_01,  # Training set mean projections
    predicted_labels  # Labels from KMeans clustering
)

print(f"Davies-Bouldin Index: {fer_sup_gabor_pca_umap_projection_10_01_db_score:.2f}")

--------

## LLE  Before UMAP

In [14]:
from sklearn.utils import resample

In [ ]:
def downsample_fer_consistent(x_data, y_labels, sample_fraction=0.75):
    """
    Downsample the dataset consistently, returning indices to ensure
    the same points are selected in both spaces.
    
    Parameters:
        x_data (np.array): Input data to downsample.
        y_labels (np.array): Corresponding labels.
        sample_fraction (float): Fraction of samples to retain per label.

    Returns:
        np.array: Array of selected indices.
    """
    sampled_indices = []
    unique_labels = np.unique(y_labels)

    for label in unique_labels:
        # Select indices for the current label
        label_indices = np.where(y_labels == label)[0]

        # Handle cases with very few samples
        n_samples = max(1, int(len(label_indices) * sample_fraction))
        if n_samples > len(label_indices):
            n_samples = len(label_indices)

        # Sample a fraction of points for this label
        sampled_indices_label = resample(
            label_indices, n_samples=n_samples, replace=False, random_state=42
        )
        sampled_indices.extend(sampled_indices_label)

    return np.array(sampled_indices)

# Downsample training data
fer_sampled_indices_train = downsample_fer_consistent(x_train_fer2013, y_train_fer2013, sample_fraction=0.75)
x_train_fer_emotions_sampled = x_train_fer2013[fer_sampled_indices_train]
y_train_fer_emotions_sampled = y_train_fer2013[fer_sampled_indices_train]

# Downsample test data
fer_sampled_indices_test = downsample_fer_consistent(x_test_fer2013, y_test_fer2013, sample_fraction=0.75)
x_test_fer_emotions_sampled = x_test_fer2013[fer_sampled_indices_test]
y_test_fer_emotions_sampled = y_test_fer2013[fer_sampled_indices_test]

# Print results
print(f"Training set reduced to {len(x_train_fer_emotions_sampled)} samples.")
print(f"Test set reduced to {len(x_test_fer_emotions_sampled)} samples.")

In [18]:
from sklearn.manifold import LocallyLinearEmbedding

In [ ]:
# Define parameters for LLE
n_neighbors = 10
n_components = 253 # Same as the reduced from PCA

# Initialize the LLE model
lle = LocallyLinearEmbedding(n_neighbors=n_neighbors, n_components=n_components, method='standard', random_state=42)

# Fit and transform the training data
print("Running LLE on the training set...")
x_train_fer_lle = lle.fit_transform(x_train_fer_emotions_sampled)
print("LLE transformation on training set completed.")

# Transform the test data using the fitted LLE model
print("Running LLE on the test set...")
x_test_fer_lle = lle.transform(x_test_fer_emotions_sampled)
print("LLE transformation on test set completed.")

# Print shapes of transformed data
print(f"Shape of LLE-transformed training data: {x_train_fer_lle.shape}")
print(f"Shape of LLE-transformed test data: {x_test_fer_lle.shape}")

# Optional: Save the LLE-transformed data for later use
np.save('x_train_lle.npy', x_train_fer_lle)
np.save('x_test_lle.npy', x_test_fer_lle)

# Output confirmation
print("LLE-transformed data has been saved.")

### LLE + UMAP Unsupervised 10 runs

In [ ]:
# Save the projections, mean, and standard deviation for the training set
fer_unsup_lle_umap_projections_train_10_01= np.load('fer_unsup_lle_umap_projections_train_10_01.npy')
fer_mean_unsup_lle_umap_projection_train_10_01= np.load('fer_mean_unsup_lle_umap_projection_train_10_01.npy')
fer_std_unsup_lle_umap_projection_train_10_01= np.load('fer_std_unsup_lle_umap_projection_train_10_01.npy')

# Save the projections, mean, and standard deviation for the test set
fer_unsup_lle_umap_projections_test_10_01= np.load('fer_unsup_lle_umap_projections_test_10_01.npy')
fer_mean_unsup_lle_umap_projection_test_10_01= np.load('fer_mean_unsup_lle_umap_projection_test_10_01.npy')
fer_std_unsup_lle_umap_projection_test_10_01= np.load('fer_std_unsup_lle_umap_projection_test_10_01.npy')

In [ ]:
# Define parameters
n_neighbors = 10
min_dist = 0.1
n_components = 2
n_runs = 10  # Number of runs

# Store UMAP projections for each run (train and test)
fer_unsup_lle_umap_projections_train_10_01 = []
fer_unsup_lle_umap_projections_test_10_01 = []

# Run UMAP multiple times for the training set
for run in range(n_runs):
    print(f"Running UMAP on Training Set - Iteration {run + 1}/{n_runs}...")
    # Create UMAP model
    umap_model = umap.UMAP(n_neighbors=n_neighbors, min_dist=min_dist, n_components=n_components, random_state=run)
    
    # Fit and transform the training data
    projection_train = umap_model.fit_transform(x_train_fer_lle)
    fer_unsup_lle_umap_projections_train_10_01.append(projection_train)
    
    # Transform the test data using the same fitted model
    print(f"Running UMAP on Test Set - Iteration {run + 1}/{n_runs}...")
    projection_test = umap_model.transform(x_test_fer_lle)
    fer_unsup_lle_umap_projections_test_10_01.append(projection_test)

# Convert the list of projections to numpy arrays
fer_unsup_lle_umap_projections_train_10_01 = np.array(fer_unsup_lle_umap_projections_train_10_01)
fer_unsup_lle_umap_projections_test_10_01 = np.array(fer_unsup_lle_umap_projections_test_10_01)

# Calculate mean and standard deviation of projections across runs (train and test)
fer_mean_unsup_lle_umap_projection_train_10_01 = np.mean(fer_unsup_lle_umap_projections_train_10_01, axis=0)
fer_std_unsup_lle_umap_projection_train_10_01 = np.std(fer_unsup_lle_umap_projections_train_10_01, axis=0)

fer_mean_unsup_lle_umap_projection_test_10_01 = np.mean(fer_unsup_lle_umap_projections_test_10_01, axis=0)
fer_std_unsup_lle_umap_projection_test_10_01 = np.std(fer_unsup_lle_umap_projections_test_10_01, axis=0)

# Save the projections, mean, and standard deviation for the training set
np.save('fer_unsup_lle_umap_projections_train_10_01.npy', fer_unsup_lle_umap_projections_train_10_01)
np.save('fer_mean_unsup_lle_umap_projection_train_10_01.npy', fer_mean_unsup_lle_umap_projection_train_10_01)
np.save('fer_std_unsup_lle_umap_projection_train_10_01.npy', fer_std_unsup_lle_umap_projection_train_10_01)

# Save the projections, mean, and standard deviation for the test set
np.save('fer_unsup_lle_umap_projections_test_10_01.npy', fer_unsup_lle_umap_projections_test_10_01)
np.save('fer_mean_unsup_lle_umap_projection_test_10_01.npy', fer_mean_unsup_lle_umap_projection_test_10_01)
np.save('fer_std_unsup_lle_umap_projection_test_10_01.npy', fer_std_unsup_lle_umap_projection_test_10_01)

# Output confirmation
print("UMAP projections for training and test sets, mean, and standard deviations have been saved.")

In [ ]:
# Adjust colormap to have exactly 7 colors
unique_labels = np.unique(y_train_fer_emotions_sampled)
cmap = plt.cm.get_cmap("tab10", len(unique_labels))

# Create the scatter plot
plt.figure(figsize=(10, 8))
scatter = plt.scatter(
    fer_mean_unsup_lle_umap_projection_train_10_01[:, 0],
    fer_mean_unsup_lle_umap_projection_train_10_01[:, 1],
    c=y_train_fer_emotions_sampled, cmap=cmap, s=5, alpha=0.8
)

# Add title and labels
plt.title("LLE + Unsupervised UMAP Projection of FER2013 Training Data (10 Runs)")
plt.xlabel("UMAP Component 1")
plt.ylabel("UMAP Component 2")

# Add and configure the colorbar
cbar = plt.colorbar(scatter)
cbar.set_ticks(range(0, 7))  # Ensure ticks align with labels
cbar.set_ticklabels([f"Emotion {label}" for label in range(0, 7)])  # Customize labels
cbar.set_label("Emotion Labels")

plt.show()

In [ ]:
# ARI
ari_fer_lle_umap_unsup_10_01 = adjusted_rand_score(y_test_fer_emotions_sampled, KNeighborsClassifier(n_neighbors=1).fit(fer_mean_unsup_lle_umap_projection_train_10_01, y_train_fer_emotions_sampled).predict(fer_mean_unsup_lle_umap_projection_test_10_01)) # second argument is y_test_pred_lle
print(f"ARI: {ari_fer_lle_umap_unsup_10_01:.2f}")
# Silhouette Score
silhouette_fer_lle_umap_unsup_10_01 = silhouette_score(fer_mean_unsup_lle_umap_projection_test_10_01, KNeighborsClassifier(n_neighbors=1).fit(fer_mean_unsup_lle_umap_projection_train_10_01, y_train_fer_emotions_sampled).predict(fer_mean_unsup_lle_umap_projection_test_10_01))
print(f"Silhouette Score: {silhouette_fer_lle_umap_unsup_10_01:.2f}")

# Use KMeans for clustering
n_clusters = len(np.unique(y_train_fer2013))  # Number of clusters = number of unique labels
kmeans = KMeans(n_clusters=n_clusters, random_state=42)

# Fit KMeans on the training set UMAP projections
predicted_labels = kmeans.fit_predict(fer_mean_unsup_lle_umap_projection_train_10_01)

# Compute Davies-Bouldin Index
fer_lle_umap_unsup_10_01_db_score = davies_bouldin_score(
    fer_mean_unsup_lle_umap_projection_train_10_01,  # Training set mean projections
    predicted_labels  # Labels from KMeans clustering
)

print(f"Davies-Bouldin Index: {fer_lle_umap_unsup_10_01_db_score:.2f}")

-----

### LLE + UMAP Supervised 10 runs

In [ ]:
# Save the projections, mean, and standard deviation for the training set
fer_sup_lle_umap_projections_train_10_01= np.load('fer_sup_lle_umap_projections_train_10_01.npy')
fer_mean_sup_lle_umap_projection_train_10_01= np.load('fer_mean_sup_lle_umap_projection_train_10_01.npy')
fer_std_sup_lle_umap_projection_train_10_01= np.load('fer_std_sup_lle_umap_projection_train_10_01.npy')

# Save the projections, mean, and standard deviation for the test set
fer_sup_lle_umap_projections_test_10_01= np.load('fer_sup_lle_umap_projections_test_10_01.npy')
fer_mean_sup_lle_umap_projection_test_10_01= np.load('fer_mean_sup_lle_umap_projection_test_10_01.npy')
fer_std_sup_lle_umap_projection_test_10_01= np.load('fer_std_sup_lle_umap_projection_test_10_01.npy')

In [ ]:
# Define parameters
n_neighbors = 10
min_dist = 0.1
n_components = 2
n_runs = 10  # Number of runs

# Store UMAP projections for each run (train and test)
fer_sup_lle_umap_projections_train_10_01 = []
fer_sup_lle_umap_projections_test_10_01 = []

# Run UMAP multiple times for the training set
for run in range(n_runs):
    print(f"Running UMAP on Training Set - Iteration {run + 1}/{n_runs}...")
    # Create UMAP model
    umap_model = umap.UMAP(n_neighbors=n_neighbors, min_dist=min_dist, n_components=n_components, random_state=run)
    
    # Fit and transform the training data
    projection_train = umap_model.fit_transform(x_train_fer_lle, y_train_fer_emotions_sampled)
    fer_sup_lle_umap_projections_train_10_01.append(projection_train)
    
    # Transform the test data using the same fitted model
    print(f"Running UMAP on Test Set - Iteration {run + 1}/{n_runs}...")
    projection_test = umap_model.transform(x_test_fer_lle)
    fer_sup_lle_umap_projections_test_10_01.append(projection_test)

# Convert the list of projections to numpy arrays
fer_sup_lle_umap_projections_train_10_01 = np.array(fer_sup_lle_umap_projections_train_10_01)
fer_sup_lle_umap_projections_test_10_01 = np.array(fer_sup_lle_umap_projections_test_10_01)

# Calculate mean and standard deviation of projections across runs (train and test)
fer_mean_sup_lle_umap_projection_train_10_01 = np.mean(fer_sup_lle_umap_projections_train_10_01, axis=0)
fer_std_sup_lle_umap_projection_train_10_01 = np.std(fer_sup_lle_umap_projections_train_10_01, axis=0)

fer_mean_sup_lle_umap_projection_test_10_01 = np.mean(fer_sup_lle_umap_projections_test_10_01, axis=0)
fer_std_sup_lle_umap_projection_test_10_01 = np.std(fer_sup_lle_umap_projections_test_10_01, axis=0)

# Save the projections, mean, and standard deviation for the training set
np.save('fer_sup_lle_umap_projections_train_10_01.npy', fer_sup_lle_umap_projections_train_10_01)
np.save('fer_mean_sup_lle_umap_projection_train_10_01.npy', fer_mean_sup_lle_umap_projection_train_10_01)
np.save('fer_std_sup_lle_umap_projection_train_10_01.npy', fer_std_sup_lle_umap_projection_train_10_01)

# Save the projections, mean, and standard deviation for the test set
np.save('fer_sup_lle_umap_projections_test_10_01.npy', fer_sup_lle_umap_projections_test_10_01)
np.save('fer_mean_sup_lle_umap_projection_test_10_01.npy', fer_mean_sup_lle_umap_projection_test_10_01)
np.save('fer_std_sup_lle_umap_projection_test_10_01.npy', fer_std_sup_lle_umap_projection_test_10_01)

# Output confirmation
print("UMAP projections for training and test sets, mean, and standard deviations have been saved.")

In [ ]:
# Adjust colormap to have exactly 7 colors
unique_labels = np.unique(y_train_fer_emotions_sampled)
cmap = plt.cm.get_cmap("tab10", len(unique_labels))

# Create the scatter plot
plt.figure(figsize=(10, 8))
scatter = plt.scatter(
    fer_mean_sup_lle_umap_projection_train_10_01[:, 0],
    fer_mean_sup_lle_umap_projection_train_10_01[:, 1],
    c=y_train_fer_emotions_sampled, cmap=cmap, s=5, alpha=0.8
)

# Add title and labels
plt.title("LLE + Supervised UMAP Projection of FER2013 Training Data (10 Runs)")
plt.xlabel("UMAP Component 1")
plt.ylabel("UMAP Component 2")

# Add and configure the colorbar
cbar = plt.colorbar(scatter)
cbar.set_ticks(range(0, 7))  # Ensure ticks align with labels
cbar.set_ticklabels([f"Emotion {label}" for label in range(0, 7)])  # Customize labels
cbar.set_label("Emotion Labels")

plt.show()

In [ ]:
# ARI
ari_fer_lle_umap_sup_10_01 = adjusted_rand_score(y_test_fer_emotions_sampled, KNeighborsClassifier(n_neighbors=1).fit(fer_mean_sup_lle_umap_projection_train_10_01, y_train_fer_emotions_sampled).predict(fer_mean_sup_lle_umap_projection_test_10_01)) # second argument is y_test_pred_lle
print(f"ARI: {ari_fer_lle_umap_sup_10_01:.2f}")
# Silhouette Score
silhouette_fer_lle_umap_sup_10_01 = silhouette_score(fer_mean_sup_lle_umap_projection_test_10_01, KNeighborsClassifier(n_neighbors=1).fit(fer_mean_sup_lle_umap_projection_train_10_01, y_train_fer_emotions_sampled).predict(fer_mean_sup_lle_umap_projection_test_10_01))
print(f"Silhouette Score: {silhouette_fer_lle_umap_sup_10_01:.2f}")

# Use KMeans for clustering
n_clusters = len(np.unique(y_train_fer2013))  # Number of clusters = number of unique labels
kmeans = KMeans(n_clusters=n_clusters, random_state=42)

# Fit KMeans on the training set UMAP projections
predicted_labels = kmeans.fit_predict(fer_mean_sup_lle_umap_projection_train_10_01)

# Compute Davies-Bouldin Index
fer_lle_umap_sup_10_01_db_score = davies_bouldin_score(
    fer_mean_sup_lle_umap_projection_train_10_01,  # Training set mean projections
    predicted_labels  # Labels from KMeans clustering
)

print(f"Davies-Bouldin Index: {fer_lle_umap_sup_10_01_db_score:.2f}")

------------

# RAF_FER

In [ ]:
# Step 1: Apply PCA to reduce dimensionality
pca = PCA(0.95)
x_train_fer2013_pca = pca.fit_transform(x_train_fer2013_scaled)  # x_train is the normalized flattened pixel data

In [ ]:
# Define parameters
n_neighbors = 10
min_dist = 0.1
n_components = 2
n_runs = 10  # Number of runs

# Store UMAP projections for each run
RAF_FER_pca_unsup_umap_projections_10_01 = []

# Run UMAP multiple times
for run in range(n_runs):
    # Create UMAP model
    umap_model = umap.UMAP(n_neighbors=n_neighbors, min_dist=min_dist, n_components=n_components,random_state=None)
    
    # Fit and transform the data
    RAF_FER_pca_umap_projection = umap_model.fit_transform(x_train_fer2013_pca)
    
    # Store the projection
    RAF_FER_pca_unsup_umap_projections_10_01.append(RAF_FER_pca_umap_projection)

# Convert the list of projections to a numpy array
RAF_FER_pca_unsup_umap_projections_10_01 = np.array(RAF_FER_pca_unsup_umap_projections_10_01)

# Calculate mean and standard deviation of projections across runs
RAF_FER_mean_pca_unsup_umap_projections_10_01 = np.mean(RAF_FER_pca_unsup_umap_projections_10_01, axis=0)
RAF_FER_std_pca_unsup_umap_projections_10_01 = np.std(RAF_FER_pca_unsup_umap_projections_10_01, axis=0)

# Save the projections, mean, and standard deviation
np.save('RAF_FER_pca_unsup_umap_projections_10_01.npy', RAF_FER_pca_unsup_umap_projections_10_01)
np.save('RAF_FER_mean_pca_unsup_umap_projections_10_01.npy', RAF_FER_mean_pca_unsup_umap_projections_10_01)
np.save('RAF_FER_std_pca_unsup_umap_projections_10_01.npy', RAF_FER_std_pca_unsup_umap_projections_10_01)

# Output confirmation
print("UMAP projections, mean, and standard deviation have been saved with identifiers '_10_01'.")

In [2]:
RAF_FER_pca_unsup_umap_projections_10_01= np.load(f'RAF_FER_pca_unsup_umap_projections_10_01.npy')
RAF_FER_mean_pca_unsup_umap_projections_10_01= np.load(f'RAF_FER_mean_pca_unsup_umap_projections_10_01.npy')
RAF_FER_std_pca_unsup_umap_projections_10_01= np.load(f'RAF_FER_std_pca_unsup_umap_projections_10_01.npy')

In [ ]:
# Visualize the UMAP results
plt.figure(figsize=(10, 8))
plt.scatter(RAF_FER_mean_pca_unsup_umap_projections_10_01[:, 0], RAF_FER_mean_pca_unsup_umap_projections_10_01[:, 1], c=y_train_fer2013, cmap="coolwarm", s=5, alpha=0.8)
plt.title("Unsup. PCA UMAP Projection: Dataset Identification FER2013")
plt.xlabel("UMAP Component 1")
plt.ylabel("UMAP Component 2")
plt.colorbar(label="FER2013 Emotions")
plt.show()

In [ ]:
# Calculate Silhouette Score
FER_pca_unsup_umap_sil_score = silhouette_score(RAF_FER_mean_pca_unsup_umap_projections_10_01, y_train_fer2013)
print(f"Silhouette Score: {FER_pca_unsup_umap_sil_score:.2f}")
# Calculate Davies-Bouldin Index
FER_pca_unsup_db_score = davies_bouldin_score(RAF_FER_mean_pca_unsup_umap_projections_10_01, y_train_fer2013)
print(f"Davies-Bouldin Index: {FER_pca_unsup_db_score:.2f}")

In [ ]:
## NOT RUN YET ##

from scipy.stats import multivariate_normal

# Fit Gaussian distributions to each class
unique_labels = np.unique(y_train)
gaussian_params = {}

for label in unique_labels:
    class_points = gabor_predicted_labels[y_train == label]
    mean = np.mean(class_points, axis=0)
    cov = np.cov(class_points, rowvar=False)
    gaussian_params[label] = multivariate_normal(mean=mean, cov=cov)

# Example: Compare likelihoods of a random point belonging to each class
sample_point = gabor_predicted_labels[0]  # Random point
likelihoods = {label: dist.pdf(sample_point) for label, dist in gaussian_params.items()}
print("Likelihoods for Random Point:", likelihoods)


In [ ]:
# Step 2: Apply UMAP to PCA-reduced data
reducer_pca = umap.UMAP(n_neighbors=50, min_dist=0.0, n_components=2, random_state=42)
x_train_umap_pca = reducer_pca.fit_transform(x_train_pca)
pca_mean_projection

In [ ]:
# Step 3: Visualize the results
plt.figure(figsize=(10, 8))
plt.scatter(
    x_train_umap_pca[:, 0],
    x_train_umap_pca[:, 1],
    c=y_train,  # 0 for FER2013, 1 for RAF
    cmap="coolwarm",
    s=5,
    alpha=0.8
)
plt.title("PCA + UMAP Projection: Dataset Identification (RAF vs FER2013)")
plt.xlabel("UMAP Component 1")
plt.ylabel("UMAP Component 2")
plt.colorbar(label="Dataset (0=FER2013, 1=RAF)")
plt.show()

In [ ]:
# Calculate silhouette score
sil_score_pca = silhouette_score(x_train_umap_pca, y_train)
print("Silhouette Score:", sil_score_pca)

## RESNET50 + UMAP

In [84]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input

In [85]:
# Step 1: Prepare Data for ResNet50
x_train_resized = np.repeat(x_train.reshape(-1, 48, 48, 1), 3, axis=-1)  # Convert grayscale to RGB
x_test_resized = np.repeat(x_test.reshape(-1, 48, 48, 1), 3, axis=-1)

In [ ]:
# Step 2: Extract Features Using ResNet50
resnet = ResNet50(weights="imagenet", include_top=False, input_shape=(48, 48, 3))

# Preprocess images
x_train_preprocessed = preprocess_input(x_train_resized)
x_test_preprocessed = preprocess_input(x_test_resized)

# Predict and extract features
train_features = resnet.predict(x_train_preprocessed, verbose=1)
test_features = resnet.predict(x_test_preprocessed, verbose=1)

# Flatten the extracted features
x_train_features = train_features.reshape(train_features.shape[0], -1)
x_test_features = test_features.reshape(test_features.shape[0], -1)

In [ ]:
# Step 3: Apply UMAP to ResNet Features
reducer3 = umap.UMAP(n_neighbors=50, min_dist=0.0, n_components=2, random_state=42)
x_train_umap3 = reducer3.fit_transform(x_train_features)

In [ ]:
# Step 4: Visualize the Results
plt.figure(figsize=(10, 8))
plt.scatter(
    x_train_umap3[:, 0],
    x_train_umap3[:, 1],
    c=y_train,  # 0 for FER2013, 1 for RAF
    cmap="coolwarm",
    s=5,
    alpha=0.8
)
plt.title("ResNet50 Features + UMAP: Dataset Identification (RAF vs FER2013)")
plt.xlabel("UMAP Component 1")
plt.ylabel("UMAP Component 2")
plt.colorbar(label="Dataset (0=FER2013, 1=RAF)")
plt.show()

In [ ]:
# Calculate silhouette score
sil_score_resnet = silhouette_score(x_train_umap3, y_train)
print("Silhouette Score:", sil_score_resnet)

In [92]:
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score

In [ ]:
# ARI METRIC
# Step 2: Cluster the UMAP-reduced data using KMeans
n_clusters = len(np.unique(y_train))  # Number of unique labels in y_train
kmeans = KMeans(n_clusters=n_clusters, random_state=42)
kmeans_labels = kmeans.fit_predict(x_train_umap3)

In [ ]:
# Step 3: Compute Adjusted Rand Index (ARI)
ari_score = adjusted_rand_score(y_train, kmeans_labels)  # y_train: ground truth labels
print("Adjusted Rand Index (ARI):", ari_score)

In [ ]:
# Step 4: Visualize Clustering Results
plt.figure(figsize=(10, 8))
plt.scatter(
    x_train_umap[:, 0],
    x_train_umap[:, 1],
    c=kmeans_labels,  # Color by KMeans clusters
    cmap="tab10",
    s=5,
    alpha=0.8
)
plt.title("UMAP + KMeans Clustering")
plt.xlabel("UMAP Component 1")
plt.ylabel("UMAP Component 2")
plt.colorbar(label="Cluster Labels")
plt.show()

-----------

## ISOMAP + UMAP

In [1]:
from sklearn.utils import resample
from sklearn.manifold import Isomap

In [ ]:
# Step 1: Downsample the Dataset Consistently
def downsample_consistent(x_train, y_train, sample_fraction=0.35):
    """
    Downsample the dataset consistently, ensuring label distribution is preserved.
    Returns sampled indices to extract data points and labels.
    """
    sampled_indices = []
    unique_labels = np.unique(y_train)
    for label in unique_labels:
        # Get indices for the current label
        label_indices = np.where(y_train == label)[0]
        # Sample a fraction of points for this label
        sampled_indices_label = resample(
            label_indices, n_samples=int(len(label_indices) * sample_fraction), replace=False
        )
        sampled_indices.extend(sampled_indices_label)
    return np.array(sampled_indices)

# Downsample x_train and y_train
sample_fraction = 0.35
sampled_indices = downsample_consistent(x_train, y_train, sample_fraction=sample_fraction)

# Extract downsampled data
x_train_sampled = x_train[sampled_indices]
y_train_sampled = y_train[sampled_indices]

print("Downsampled x_train shape:", x_train_sampled.shape)
print("Downsampled y_train shape:", y_train_sampled.shape)

In [ ]:
# Step 2: Normalize the Sampled Data
scaler = StandardScaler()
x_train_sampled_scaled = scaler.fit_transform(x_train_sampled)

In [ ]:
# Step 3: Apply Isomap for Dimensionality Reduction
n_neighbors_isomap = 10  # Number of neighbors for Isomap
n_components_isomap = 50  # Reduce to 50 dimensions before UMAP
isomap = Isomap(n_neighbors=n_neighbors_isomap, n_components=n_components_isomap)
x_train_isomap = isomap.fit_transform(x_train_sampled_scaled)

print("Isomap reduced shape:", x_train_isomap.shape)

# Step 4: Apply UMAP for Further Reduction to 2D
reducer = umap.UMAP(n_neighbors=15, min_dist=0.1, n_components=2, random_state=42)
x_train_umap = reducer.fit_transform(x_train_isomap)

# Step 5: Visualize the Results
plt.figure(figsize=(10, 8))
plt.scatter(
    x_train_umap[:, 0],
    x_train_umap[:, 1],
    c=y_train_sampled,  # Color by ground truth labels
    cmap="tab10",
    s=5,
    alpha=0.8
)
plt.title("Isomap + UMAP Projection: Downsampled Training Data")
plt.xlabel("UMAP Component 1")
plt.ylabel("UMAP Component 2")
plt.colorbar(label="Labels")
plt.show()

In [ ]:
# Save the sampled indices
np.save("sampled_indices.npy", sampled_indices)

# Save the downsampled dataset
np.save("x_sampled.npy", x_sampled)
np.save("y_sampled.npy", y_sampled)

print("Downsampling saved successfully!")

In [ ]:
# Apply Isomap to the subset of data
isomap = Isomap(n_components=50, n_neighbors=5)  # Reduced n_neighbors to 5 for faster computation
# x_train_isomap_subset = isomap.fit_transform(x_train_subset)
x_reduced = isomap.fit_transform(x_sampled)

In [ ]:
#Save the Isomap-reduced data
# np.save("isomap_embedding_subsets.npy", x_train_isomap_subset)
np.save("isomap_embedding_subsets_evenly_downsampled.npy", x_reduced)
print("Isomap embeddings saved.")